<a href="https://colab.research.google.com/github/thegallier/timeseries/blob/main/timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import torch.nn.functional as F
from sklearn.metrics import cohen_kappa_score
from torch.utils.tensorboard import SummaryWriter
from itertools import product
import time

# Parameters
num_timesteps = 5000  # Reduced for practical purposes
num_securities = 10
num_features_per_security = 4
num_classes = 3
num_features = num_securities * num_features_per_security

# Generate timestamps
timestamps = np.arange(num_timesteps)

# Generate random data for the primary dataset
X_data = np.random.rand(num_timesteps, num_features).astype(np.float32)
y_data = np.random.randint(0, num_classes, size=(num_timesteps, num_securities)).astype(np.int64)

# Generate the second dataset
# 3 strings and 2 floats per timestamp
str_columns = ['str1', 'str2', 'str3']
float_columns = ['float1', 'float2']
second_dataset = {
    'timestamp': timestamps,
    'str1': np.random.choice(['A', 'B', 'C'], num_timesteps),
    'str2': np.random.choice(['D', 'E', 'F'], num_timesteps),
    'str3': np.random.choice(['G', 'H', 'I'], num_timesteps),
    'float1': np.random.rand(num_timesteps),
    'float2': np.random.rand(num_timesteps),
}

# Encode string columns
label_encoders = {}
for col in str_columns:
    le = LabelEncoder()
    second_dataset[col] = le.fit_transform(second_dataset[col])
    label_encoders[col] = le

# Combine all features from the second dataset
second_X_data = np.column_stack([second_dataset[col] for col in str_columns + float_columns]).astype(np.float32)

# Min-max scaling for both datasets
scaler_X = MinMaxScaler()
X_data = scaler_X.fit_transform(X_data)

scaler_second_X = MinMaxScaler()
second_X_data = scaler_second_X.fit_transform(second_X_data)

def add_positional_encoding(X, timestamps, option='shared'):
    """
    Add positional encoding to the feature matrix.

    Parameters:
    - X: Feature matrix.
    - timestamps: Array of timestamps.
    - option: 'shared' or 'per_security'.

    Returns:
    - X with positional encoding added.
    """
    if option == 'shared':
        pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(X.shape[1]) / X.shape[1]))
        X_pe = X + pe.astype(np.float32)
    elif option == 'per_security':
        pe_list = []
        for i in range(num_securities):
            pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(num_features_per_security) / num_features_per_security))
            pe_list.append(pe)
        pe_concat = np.hstack(pe_list)
        X_pe = X + pe_concat.astype(np.float32)
    else:
        X_pe = X  # No positional encoding
    return X_pe

# Apply positional encoding
positional_encoding_option = 'shared'  # 'shared' or 'per_security'
X_data = add_positional_encoding(X_data, timestamps, positional_encoding_option)

def create_windows(X1, X2, y, window_size, horizon):
    """
    Create sliding windows for time series data.

    Parameters:
    - X1: Primary dataset features.
    - X2: Secondary dataset features.
    - y: Target variable.
    - window_size: Size of the window.
    - horizon: Prediction horizon.

    Returns:
    - Tuples of windows for X1, X2, and y.
    """
    X1_windows = []
    X2_windows = []
    y_windows = []
    for i in range(len(X1) - window_size - horizon + 1):
        X1_windows.append(X1[i:i+window_size])
        X2_windows.append(X2[i:i+window_size])
        y_windows.append(y[i+window_size+horizon-1])
    return np.array(X1_windows), np.array(X2_windows), np.array(y_windows)

# Define window sizes
window_size = 20
horizon = 1

# Create windows
X1_windows, X2_windows, y_windows = create_windows(X_data, second_X_data, y_data, window_size, horizon)

# Split into train and test sets
train_ratio = 0.8
train_size = int(len(X1_windows) * train_ratio)

X1_train = X1_windows[:train_size]
X2_train = X2_windows[:train_size]
y_train = y_windows[:train_size]

X1_test = X1_windows[train_size:]
X2_test = X2_windows[train_size:]
y_test = y_windows[train_size:]

# Convert to PyTorch tensors
X1_train_tensor = torch.tensor(X1_train)
X2_train_tensor = torch.tensor(X2_train)
y_train_tensor = torch.tensor(y_train)

X1_test_tensor = torch.tensor(X1_test)
X2_test_tensor = torch.tensor(X2_test)
y_test_tensor = torch.tensor(y_test)

class TimeSeriesDataset(Dataset):
    """
    Custom Dataset for time series data with two feature sets.

    Parameters:
    - X1: Primary dataset features.
    - X2: Secondary dataset features.
    - y: Target variable.
    """
    def __init__(self, X1, X2, y):
        self.X1 = X1.float()
        self.X2 = X2.float()
        self.y = y.long()
    def __len__(self):
        return len(self.X1)
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]

# DataLoaders
batch_size = 64

train_dataset = TimeSeriesDataset(X1_train_tensor, X2_train_tensor, y_train_tensor)
test_dataset = TimeSeriesDataset(X1_test_tensor, X2_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class MatrixRegressionModel(nn.Module):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.
    """
    def __init__(self, num_securities, num_features):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(1, num_securities))
        self.B = nn.Parameter(torch.randn(num_features, num_classes))
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten over time
        out = self.A @ x @ self.B  # Shape: (batch_size, num_classes)
        out = out.view(-1, num_securities, num_classes)
        return out

class LogisticRegressionModel(nn.Module):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.
    """
    def __init__(self, input_size, num_securities, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.linear = nn.Linear(input_size, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.linear(x)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class CNNModel(nn.Module):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(CNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch_size, num_features, window_size)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        out = x.view(-1, self.num_securities, self.num_classes)
        return out

class LSTMModel(nn.Module):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.
    """
    def __init__(self, input_size, hidden_size, num_layers, num_securities, num_classes):
        super(LSTMModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class TransformerModel(nn.Module):
    """
    Transformer Model.

    Advantages:
    - Captures global dependencies.
    - Parallel computation.

    Disadvantages:
    - Requires large datasets.
    - Computationally intensive.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(num_features, 128)
        self.pos_encoder = PositionalEncoding(128)
        encoder_layers = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=2)
        self.decoder = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), x.size(1), -1)  # (batch_size, window_size, num_features)
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embedding_dim)
        output = self.transformer_encoder(x)
        output = output[-1, :, :]  # Last output
        output = self.decoder(output)
        output = output.view(-1, self.num_securities, num_classes)
        return output

class PositionalEncoding(nn.Module):
    """
    Positional Encoding for Transformer.

    Adds positional information to the embeddings.
    """
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.sin(position * div_term)
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)
    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return x

class SimpleRNNModel(nn.Module):
    """
    Simple Recurrent Neural Network Model.

    Advantages:
    - Captures sequential dependencies.
    - Simpler than LSTM.

    Disadvantages:
    - Struggles with long-term dependencies.
    - May suffer from vanishing gradients.
    """
    def __init__(self, input_size, hidden_size, num_securities, num_classes):
        super(SimpleRNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.rnn.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class CombinedModel(nn.Module):
    """
    Combined Model: Merges outputs from two models.

    Advantages:
    - Leverages multiple data sources.
    - Potentially better performance.

    Disadvantages:
    - More complex.
    - Computationally intensive.
    """
    def __init__(self, model1, model2, num_classes, num_securities):
        super(CombinedModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.fc = nn.Linear(num_classes * 2, num_classes)
        self.num_securities = num_securities
    def forward(self, x1, x2):
        outputs1 = self.model1(x1)
        outputs2 = self.model2(x2)
        outputs = torch.cat((outputs1, outputs2), dim=2)
        batch_size = outputs.size(0)
        outputs = outputs.view(-1, outputs.size(2))
        final_output = self.fc(outputs)
        final_output = final_output.view(batch_size, self.num_securities, -1)
        return final_output

def train_model(model, train_loader, criterion, optimizer, num_epochs, device, writer):
    """
    Training loop for the model.

    Parameters:
    - model: The model to train.
    - train_loader: DataLoader for training data.
    - criterion: Loss function.
    - optimizer: Optimizer.
    - num_epochs: Number of epochs.
    - device: Computation device.
    - writer: TensorBoard SummaryWriter.
    """
    model = model.to(device)
    global_step = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X1_batch, X2_batch, y_batch in train_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X1_batch, X2_batch)
            loss = 0
            for i in range(num_securities):
                loss += criterion(outputs[:, i, :], y_batch[:, i])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            writer.add_scalar('Training Loss', loss.item(), global_step)
            global_step += 1
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
        # Save checkpoint
        torch.save(model.state_dict(), f'checkpoint_epoch_{epoch+1}.pth')

def evaluate_model(model, test_loader, device):
    """
    Evaluate the model on the test set.

    Parameters:
    - model: The model to evaluate.
    - test_loader: DataLoader for test data.
    - device: Computation device.

    Returns:
    - Cohen's kappa score.
    """
    model = model.to(device)
    model.eval()
    total = 0
    correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X1_batch, X2_batch, y_batch in test_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X1_batch, X2_batch)
            _, predicted = torch.max(outputs.data, 2)
            total += y_batch.numel()
            correct += (predicted == y_batch).sum().item()
            all_preds.extend(predicted.cpu().numpy().flatten())
            all_targets.extend(y_batch.cpu().numpy().flatten())
    accuracy = 100 * correct / total
    kappa = cohen_kappa_score(all_targets, all_preds)
    print(f'Accuracy on test set: {accuracy:.2f}%')
    print(f"Cohen's Kappa: {kappa:.4f}")
    return kappa

def hyperparameter_tuning(hyperparams, train_loader, test_loader, device):
    """
    Perform hyperparameter tuning.

    Parameters:
    - hyperparams: Dictionary of hyperparameters to try.
    - train_loader: DataLoader for training data.
    - test_loader: DataLoader for test data.
    - device: Computation device.
    """
    best_kappa = -1
    best_params = None
    for params in product(*hyperparams.values()):
        param_dict = dict(zip(hyperparams.keys(), params))
        print(f"Trying hyperparameters: {param_dict}")
        model1 = TransformerModel(num_features=num_features, num_securities=num_securities, num_classes=num_classes)
        model2 = LSTMModel(input_size=second_X_data.shape[1], hidden_size=param_dict['hidden_size'],
                           num_layers=param_dict['num_layers'], num_securities=num_securities, num_classes=num_classes)
        combined_model = CombinedModel(model1, model2, num_classes, num_securities)
        optimizer = torch.optim.Adam(combined_model.parameters(), lr=param_dict['learning_rate'])
        criterion = nn.CrossEntropyLoss()
        writer = SummaryWriter()
        train_model(combined_model, train_loader, criterion, optimizer, param_dict['num_epochs'], device, writer)
        kappa = evaluate_model(combined_model, test_loader, device)
        writer.close()
        if kappa > best_kappa:
            best_kappa = kappa
            best_params = param_dict
    print(f"Best Cohen's Kappa: {best_kappa:.4f} with parameters: {best_params}")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters for tuning
hyperparams = {
    'hidden_size': [64, 128],
    'num_layers': [1, 2],
    'learning_rate': [0.001, 0.0001],
    'num_epochs': [5]
}

# Start hyperparameter tuning
hyperparameter_tuning(hyperparams, train_loader, test_loader, device)

model = LSTMModel(input_size=num_features, hidden_size=128, num_layers=2, num_securities=num_securities, num_classes=num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)

evaluate_model(model, test_loader, device)
writer.close()

Trying hyperparameters: {'hidden_size': 64, 'num_layers': 1, 'learning_rate': 0.001, 'num_epochs': 5}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


AttributeError: 'TransformerModel' object has no attribute 'num_securities'

In [17]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import torch.nn.functional as F
from sklearn.metrics import cohen_kappa_score
from torch.utils.tensorboard import SummaryWriter
from itertools import product
import time

# Parameters
num_timesteps = 5000  # Reduced for practical purposes
num_securities = 10  # Number of different securities
num_features_per_security = 4  # Features per security
num_classes = 3  # Number of target classes
num_features = num_securities * num_features_per_security  # Total number of features

# Generate timestamps
timestamps = np.arange(num_timesteps)

# Generate random data for the primary dataset (X_data) and labels (y_data)
X_data = np.random.rand(num_timesteps, num_features).astype(np.float32)  # Random features
y_data = np.random.randint(0, num_classes, size=(num_timesteps, num_securities)).astype(np.int64)  # Random labels

# Generate the second dataset with 3 categorical strings and 2 continuous floats per timestamp
str_columns = ['str1', 'str2', 'str3']
float_columns = ['float1', 'float2']
second_dataset = {
    'timestamp': timestamps,
    'str1': np.random.choice(['A', 'B', 'C'], num_timesteps),
    'str2': np.random.choice(['D', 'E', 'F'], num_timesteps),
    'str3': np.random.choice(['G', 'H', 'I'], num_timesteps),
    'float1': np.random.rand(num_timesteps),
    'float2': np.random.rand(num_timesteps),
}

# Encode categorical string columns using LabelEncoder
label_encoders = {}
for col in str_columns:
    le = LabelEncoder()
    second_dataset[col] = le.fit_transform(second_dataset[col])
    label_encoders[col] = le

# Combine all features from the second dataset into a single array
second_X_data = np.column_stack([second_dataset[col] for col in str_columns + float_columns]).astype(np.float32)

# Apply Min-Max scaling to both datasets
scaler_X = MinMaxScaler()
X_data = scaler_X.fit_transform(X_data)

scaler_second_X = MinMaxScaler()
second_X_data = scaler_second_X.fit_transform(second_X_data)

def add_positional_encoding(X, timestamps, option='shared'):
    """
    Add positional encoding to the feature matrix.

    Parameters:
    - X (ndarray): Feature matrix.
    - timestamps (ndarray): Array of timestamps.
    - option (str): 'shared' or 'per_security'.

    Returns:
    - X_pe (ndarray): X with positional encoding added.
    """
    if option == 'shared':
        # Shared positional encoding across all features
        pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(X.shape[1]) / X.shape[1]))
        X_pe = X + pe.astype(np.float32)
    elif option == 'per_security':
        # Separate positional encoding for each security
        pe_list = []
        for i in range(num_securities):
            pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(num_features_per_security) / num_features_per_security))
            pe_list.append(pe)
        pe_concat = np.hstack(pe_list)
        X_pe = X + pe_concat.astype(np.float32)
    else:
        X_pe = X  # No positional encoding
    return X_pe

# Apply positional encoding
positional_encoding_option = 'shared'  # 'shared' or 'per_security'
X_data = add_positional_encoding(X_data, timestamps, positional_encoding_option)

def create_windows(X1, X2, y, window_size, horizon):
    """
    Create sliding windows for time series data.

    Parameters:
    - X1 (ndarray): Primary dataset features.
    - X2 (ndarray): Secondary dataset features.
    - y (ndarray): Target variable.
    - window_size (int): Size of the window.
    - horizon (int): Prediction horizon.

    Returns:
    - Tuple[np.ndarray, np.ndarray, np.ndarray]: Tuples of windows for X1, X2, and y.
    """
    X1_windows = []
    X2_windows = []
    y_windows = []
    for i in range(len(X1) - window_size - horizon + 1):
        X1_windows.append(X1[i:i+window_size])
        X2_windows.append(X2[i:i+window_size])
        y_windows.append(y[i+window_size+horizon-1])
    return np.array(X1_windows), np.array(X2_windows), np.array(y_windows)

# Define window sizes
window_size = 20
horizon = 1

# Create windows
X1_windows, X2_windows, y_windows = create_windows(X_data, second_X_data, y_data, window_size, horizon)

# Split into train and test sets
train_ratio = 0.8
train_size = int(len(X1_windows) * train_ratio)

X1_train = X1_windows[:train_size]
X2_train = X2_windows[:train_size]
y_train = y_windows[:train_size]

X1_test = X1_windows[train_size:]
X2_test = X2_windows[train_size:]
y_test = y_windows[train_size:]

# Convert to PyTorch tensors
X1_train_tensor = torch.tensor(X1_train)
X2_train_tensor = torch.tensor(X2_train)
y_train_tensor = torch.tensor(y_train)

X1_test_tensor = torch.tensor(X1_test)
X2_test_tensor = torch.tensor(X2_test)
y_test_tensor = torch.tensor(y_test)

class TimeSeriesDataset(Dataset):
    """
    Custom Dataset for time series data with two feature sets.

    Parameters:
    - X1 (torch.Tensor): Primary dataset features.
    - X2 (torch.Tensor): Secondary dataset features.
    - y (torch.Tensor): Target variable.
    """
    def __init__(self, X1, X2, y):
        self.X1 = X1.float()
        self.X2 = X2.float()
        self.y = y.long()
    def __len__(self):
        return len(self.X1)
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]

# DataLoaders
batch_size = 64

train_dataset = TimeSeriesDataset(X1_train_tensor, X2_train_tensor, y_train_tensor)
test_dataset = TimeSeriesDataset(X1_test_tensor, X2_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class MatrixRegressionModel(nn.Module):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.
    """
    def __init__(self, num_securities, num_features):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(1, num_securities))
        self.B = nn.Parameter(torch.randn(num_features, num_classes))
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten over time
        out = self.A @ x @ self.B  # Shape: (batch_size, num_classes)
        out = out.view(-1, num_securities, num_classes)
        return out

class LogisticRegressionModel(nn.Module):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.
    """
    def __init__(self, input_size, num_securities, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.linear = nn.Linear(input_size, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.linear(x)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class CNNModel(nn.Module):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(CNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch_size, num_features, window_size)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        out = x.view(-1, self.num_securities, self.num_classes)
        return out

class LSTMModel(nn.Module):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.
    """
    def __init__(self, input_size, hidden_size, num_layers, num_securities, num_classes):
        super(LSTMModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class TransformerModel(nn.Module):
    """
    Transformer Model.

    Advantages:
    - Captures global dependencies.
    - Parallel computation.

    Disadvantages:
    - Requires large datasets.
    - Computationally intensive.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(TransformerModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.embedding = nn.Linear(num_features, 128)
        self.pos_encoder = PositionalEncoding(128)
        encoder_layers = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=2)
        self.decoder = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), x.size(1), -1)  # (batch_size, window_size, num_features)
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embedding_dim)
        output = self.transformer_encoder(x)
        output = output[-1, :, :]  # Last output
        output = self.decoder(output)
        output = output.view(-1, self.num_securities, self.num_classes)
        return output

class PositionalEncoding(nn.Module):
    """
    Positional Encoding for Transformer.

    Adds positional information to the embeddings.
    """
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.sin(position * div_term)
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)
    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return x

class SimpleRNNModel(nn.Module):
    """
    Simple Recurrent Neural Network Model.

    Advantages:
    - Captures sequential dependencies.
    - Simpler than LSTM.

    Disadvantages:
    - Struggles with long-term dependencies.
    - May suffer from vanishing gradients.
    """
    def __init__(self, input_size, hidden_size, num_securities, num_classes):
        super(SimpleRNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.rnn.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class CombinedModel(nn.Module):
    """
    Combined Model: Merges outputs from two models.

    Advantages:
    - Leverages multiple data sources.
    - Potentially better performance.

    Disadvantages:
    - More complex.
    - Computationally intensive.
    """
    def __init__(self, model1, model2, num_classes, num_securities):
        super(CombinedModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.fc = nn.Linear(num_classes * 2, num_classes)
        self.num_securities = num_securities
        self.num_classes = num_classes
    def forward(self, x1, x2):
        outputs1 = self.model1(x1)
        outputs2 = self.model2(x2)
        outputs = torch.cat((outputs1, outputs2), dim=2)
        batch_size = outputs.size(0)
        outputs = outputs.view(-1, outputs.size(2))
        final_output = self.fc(outputs)
        final_output = final_output.view(batch_size, self.num_securities, -1)
        return final_output

def train_model(model, train_loader, criterion, optimizer, num_epochs, device, writer):
    """
    Training loop for the model.

    Parameters:
    - model (nn.Module): The model to train.
    - train_loader (DataLoader): DataLoader for training data.
    - criterion (nn.Module): Loss function.
    - optimizer (torch.optim.Optimizer): Optimizer.
    - num_epochs (int): Number of epochs.
    - device (torch.device): Computation device.
    - writer (SummaryWriter): TensorBoard SummaryWriter.
    """
    model = model.to(device)
    global_step = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X1_batch, X2_batch, y_batch in train_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X1_batch, X2_batch)
            loss = 0
            for i in range(num_securities):
                loss += criterion(outputs[:, i, :], y_batch[:, i])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            writer.add_scalar('Training Loss', loss.item(), global_step)
            global_step += 1
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
        # Save checkpoint
        torch.save(model.state_dict(), f'checkpoint_epoch_{epoch+1}.pth')

def evaluate_model(model, test_loader, device):
    """
    Evaluate the model on the test set.

    Parameters:
    - model (nn.Module): The model to evaluate.
    - test_loader (DataLoader): DataLoader for test data.
    - device (torch.device): Computation device.

    Returns:
    - float: Cohen's kappa score.
    """
    model = model.to(device)
    model.eval()
    total = 0
    correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X1_batch, X2_batch, y_batch in test_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X1_batch, X2_batch)
            _, predicted = torch.max(outputs.data, 2)
            total += y_batch.numel()
            correct += (predicted == y_batch).sum().item()
            all_preds.extend(predicted.cpu().numpy().flatten())
            all_targets.extend(y_batch.cpu().numpy().flatten())
    accuracy = 100 * correct / total
    kappa = cohen_kappa_score(all_targets, all_preds)
    print(f'Accuracy on test set: {accuracy:.2f}%')
    print(f"Cohen's Kappa: {kappa:.4f}")
    return kappa

def hyperparameter_tuning(hyperparams, train_loader, test_loader, device):
    """
    Perform hyperparameter tuning.

    Parameters:
    - hyperparams (dict): Dictionary of hyperparameters to try.
    - train_loader (DataLoader): DataLoader for training data.
    - test_loader (DataLoader): DataLoader for test data.
    - device (torch.device): Computation device.
    """
    best_kappa = -1
    best_params = None
    for params in product(*hyperparams.values()):
        param_dict = dict(zip(hyperparams.keys(), params))
        print(f"Trying hyperparameters: {param_dict}")
        model1 = TransformerModel(num_features=num_features, num_securities=num_securities, num_classes=num_classes)
        model2 = LSTMModel(input_size=second_X_data.shape[1], hidden_size=param_dict['hidden_size'],
                           num_layers=param_dict['num_layers'], num_securities=num_securities, num_classes=num_classes)
        combined_model = CombinedModel(model1, model2, num_classes, num_securities)
        optimizer = torch.optim.Adam(combined_model.parameters(), lr=param_dict['learning_rate'])
        criterion = nn.CrossEntropyLoss()
        writer = SummaryWriter()
        train_model(combined_model, train_loader, criterion, optimizer, param_dict['num_epochs'], device, writer)
        kappa = evaluate_model(combined_model, test_loader, device)
        writer.close()
        if kappa > best_kappa:
            best_kappa = kappa
            best_params = param_dict
    print(f"Best Cohen's Kappa: {best_kappa:.4f} with parameters: {best_params}")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters for tuning
hyperparams = {
    'hidden_size': [64, 128],
    'num_layers': [1, 2],
    'learning_rate': [0.001, 0.0001],
    'num_epochs': [5]
}

# Start hyperparameter tuning
hyperparameter_tuning(hyperparams, train_loader, test_loader, device)

# Example 1: Combine TransformerModel and LSTMModel
print("\nTraining Combined Model 1: TransformerModel + LSTMModel")
model1 = TransformerModel(num_features=num_features, num_securities=num_securities, num_classes=num_classes)
model2 = LSTMModel(input_size=second_X_data.shape[1], hidden_size=128, num_layers=2, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 2: Combine CNNModel and SimpleRNNModel
print("\nTraining Combined Model 2: CNNModel + SimpleRNNModel")
model1 = CNNModel(num_features=num_features, num_securities=num_securities, num_classes=num_classes)
model2 = SimpleRNNModel(input_size=second_X_data.shape[1], hidden_size=64, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 3: Combine LogisticRegressionModel and LSTMModel
print("\nTraining Combined Model 3: LogisticRegressionModel + LSTMModel")
model1 = LogisticRegressionModel(input_size=num_features*window_size, num_securities=num_securities, num_classes=num_classes)
model2 = LSTMModel(input_size=second_X_data.shape[1], hidden_size=128, num_layers=1, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

Trying hyperparameters: {'hidden_size': 64, 'num_layers': 1, 'learning_rate': 0.001, 'num_epochs': 5}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: 11.0394
Epoch [2/5], Loss: 11.0018
Epoch [3/5], Loss: 10.9970
Epoch [4/5], Loss: 10.9921
Epoch [5/5], Loss: 10.9896
Accuracy on test set: 33.48%
Cohen's Kappa: -0.0003
Trying hyperparameters: {'hidden_size': 64, 'num_layers': 1, 'learning_rate': 0.0001, 'num_epochs': 5}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: 11.0810
Epoch [2/5], Loss: 11.0141
Epoch [3/5], Loss: 11.0052
Epoch [4/5], Loss: 10.9961
Epoch [5/5], Loss: 10.9905
Accuracy on test set: 33.35%
Cohen's Kappa: 0.0000
Trying hyperparameters: {'hidden_size': 64, 'num_layers': 2, 'learning_rate': 0.001, 'num_epochs': 5}


...

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: 11.0756
Epoch [2/5], Loss: 11.0242
Epoch [3/5], Loss: 11.0151
Epoch [4/5], Loss: 11.0062
Epoch [5/5], Loss: 11.0032
Accuracy on test set: 33.87%
Cohen's Kappa: 0.0108
Trying hyperparameters: {'hidden_size': 128, 'num_layers': 2, 'learning_rate': 0.0001, 'num_epochs': 5}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: 11.0780
Epoch [2/5], Loss: 11.0124
Epoch [3/5], Loss: 10.9964


KeyboardInterrupt: 

In [20]:
### added more models
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import cohen_kappa_score
from torch.utils.tensorboard import SummaryWriter
from itertools import product
import time

# Parameters
num_timesteps = 5000  # Reduced for practical purposes
num_securities = 10  # Number of different securities
num_features_per_security = 4  # Features per security
num_classes = 3  # Number of target classes
num_features = num_securities * num_features_per_security  # Total number of features

# Generate timestamps
timestamps = np.arange(num_timesteps)

# Generate random data for the primary dataset (X_data) and labels (y_data)
X_data = np.random.rand(num_timesteps, num_features).astype(np.float32)  # Random features
y_data = np.random.randint(0, num_classes, size=(num_timesteps, num_securities)).astype(np.int64)  # Random labels

# Generate the second dataset with 3 categorical strings and 2 continuous floats per timestamp
str_columns = ['str1', 'str2', 'str3']
float_columns = ['float1', 'float2']
second_dataset = {
    'timestamp': timestamps,
    'str1': np.random.choice(['A', 'B', 'C'], num_timesteps),
    'str2': np.random.choice(['D', 'E', 'F'], num_timesteps),
    'str3': np.random.choice(['G', 'H', 'I'], num_timesteps),
    'float1': np.random.rand(num_timesteps),
    'float2': np.random.rand(num_timesteps),
}

# Encode categorical string columns using LabelEncoder
label_encoders = {}
for col in str_columns:
    le = LabelEncoder()
    second_dataset[col] = le.fit_transform(second_dataset[col])
    label_encoders[col] = le

# Combine all features from the second dataset into a single array
second_X_data = np.column_stack([second_dataset[col] for col in str_columns + float_columns]).astype(np.float32)

# Apply Min-Max scaling to both datasets
scaler_X = MinMaxScaler()
X_data = scaler_X.fit_transform(X_data)

scaler_second_X = MinMaxScaler()
second_X_data = scaler_second_X.fit_transform(second_X_data)

def add_positional_encoding(X, timestamps, option='shared'):
    """
    Add positional encoding to the feature matrix.

    Parameters:
    - X (ndarray): Feature matrix.
    - timestamps (ndarray): Array of timestamps.
    - option (str): 'shared' or 'per_security'.

    Returns:
    - X_pe (ndarray): X with positional encoding added.
    """
    if option == 'shared':
        # Shared positional encoding across all features
        pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(X.shape[1]) / X.shape[1]))
        X_pe = X + pe.astype(np.float32)
    elif option == 'per_security':
        # Separate positional encoding for each security
        pe_list = []
        for i in range(num_securities):
            pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(num_features_per_security) / num_features_per_security))
            pe_list.append(pe)
        pe_concat = np.hstack(pe_list)
        X_pe = X + pe_concat.astype(np.float32)
    else:
        X_pe = X  # No positional encoding
    return X_pe

# Apply positional encoding
positional_encoding_option = 'shared'  # 'shared' or 'per_security'
X_data = add_positional_encoding(X_data, timestamps, positional_encoding_option)

def create_windows(X1, X2, y, window_size, horizon):
    """
    Create sliding windows for time series data.

    Parameters:
    - X1 (ndarray): Primary dataset features.
    - X2 (ndarray): Secondary dataset features.
    - y (ndarray): Target variable.
    - window_size (int): Size of the window.
    - horizon (int): Prediction horizon.

    Returns:
    - Tuple[np.ndarray, np.ndarray, np.ndarray]: Tuples of windows for X1, X2, and y.
    """
    X1_windows = []
    X2_windows = []
    y_windows = []
    for i in range(len(X1) - window_size - horizon + 1):
        X1_windows.append(X1[i:i+window_size])
        X2_windows.append(X2[i:i+window_size])
        y_windows.append(y[i+window_size+horizon-1])
    return np.array(X1_windows), np.array(X2_windows), np.array(y_windows)

# Define window sizes
window_size = 20
horizon = 1

# Create windows
X1_windows, X2_windows, y_windows = create_windows(X_data, second_X_data, y_data, window_size, horizon)

# Split into train and test sets
train_ratio = 0.8
train_size = int(len(X1_windows) * train_ratio)

X1_train = X1_windows[:train_size]
X2_train = X2_windows[:train_size]
y_train = y_windows[:train_size]

X1_test = X1_windows[train_size:]
X2_test = X2_windows[train_size:]
y_test = y_windows[train_size:]

# Convert to PyTorch tensors
X1_train_tensor = torch.tensor(X1_train)
X2_train_tensor = torch.tensor(X2_train)
y_train_tensor = torch.tensor(y_train)

X1_test_tensor = torch.tensor(X1_test)
X2_test_tensor = torch.tensor(X2_test)
y_test_tensor = torch.tensor(y_test)

class TimeSeriesDataset(Dataset):
    """
    Custom Dataset for time series data with two feature sets.

    Parameters:
    - X1 (torch.Tensor): Primary dataset features.
    - X2 (torch.Tensor): Secondary dataset features.
    - y (torch.Tensor): Target variable.
    """
    def __init__(self, X1, X2, y):
        self.X1 = X1.float()
        self.X2 = X2.float()
        self.y = y.long()
    def __len__(self):
        return len(self.X1)
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]

# DataLoaders
batch_size = 64

train_dataset = TimeSeriesDataset(X1_train_tensor, X2_train_tensor, y_train_tensor)
test_dataset = TimeSeriesDataset(X1_test_tensor, X2_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class MatrixRegressionModel(nn.Module):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.

    Parameters:
    - num_securities (int): Number of securities.
    - num_features (int): Total number of features.
    """
    def __init__(self, num_securities, num_features):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(1, num_securities))
        self.B = nn.Parameter(torch.randn(num_features, num_classes))
        self.num_securities = num_securities
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten over time
        out = self.A @ x @ self.B  # Shape: (batch_size, num_classes)
        out = out.view(-1, self.num_securities, num_classes)
        return out

class LogisticRegressionModel(nn.Module):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.

    Parameters:
    - input_size (int): Size of input features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, num_securities, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.linear = nn.Linear(input_size, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.linear(x)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class CNNModel(nn.Module):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(CNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch_size, num_features, window_size)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        out = x.view(-1, self.num_securities, self.num_classes)
        return out

class LSTMModel(nn.Module):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_layers (int): Number of LSTM layers.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_layers, num_securities, num_classes):
        super(LSTMModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class TransformerModel(nn.Module):
    """
    Transformer Model.

    Advantages:
    - Captures global dependencies.
    - Parallel computation.

    Disadvantages:
    - Requires large datasets.
    - Computationally intensive.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(TransformerModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.embedding = nn.Linear(num_features, 128)
        self.pos_encoder = PositionalEncoding(128)
        encoder_layers = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=2)
        self.decoder = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), x.size(1), -1)  # (batch_size, window_size, num_features)
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embedding_dim)
        output = self.transformer_encoder(x)
        output = output[-1, :, :]  # Last output
        output = self.decoder(output)
        output = output.view(-1, self.num_securities, self.num_classes)
        return output

class PositionalEncoding(nn.Module):
    """
    Positional Encoding for Transformer.

    Adds positional information to the embeddings.

    Parameters:
    - d_model (int): Embedding dimension.
    - max_len (int): Maximum sequence length.
    """
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.sin(position * div_term)
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)
    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return x

class SimpleRNNModel(nn.Module):
    """
    Simple Recurrent Neural Network Model.

    Advantages:
    - Captures sequential dependencies.
    - Simpler than LSTM.

    Disadvantages:
    - Struggles with long-term dependencies.
    - May suffer from vanishing gradients.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_securities, num_classes):
        super(SimpleRNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

# Missing Models with Docstrings and Comments

class GNNModel(nn.Module):
    """
    Graph Neural Network Model.

    Advantages:
    - Captures relationships between securities.
    - Utilizes attention mechanisms.

    Disadvantages:
    - Computationally intensive.
    - Requires graph structure.

    Parameters:
    - num_securities (int): Number of securities.
    - num_features_per_security (int): Features per security.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_securities, num_features_per_security, num_classes):
        super(GNNModel, self).__init__()
        self.num_securities = num_securities
        # Create a linear layer for each security
        self.gcn_layers = nn.ModuleList([nn.Linear(num_features_per_security, 64) for _ in range(num_securities)])
        # Multi-head attention mechanism
        self.attention = nn.MultiheadAttention(embed_dim=64, num_heads=4, batch_first=True)
        # Fully connected layer for classification
        self.fc = nn.Linear(64, num_classes)
    def forward(self, x):
        # x shape: (batch_size, window_size, num_features)
        batch_size = x.size(0)
        x = x[:, -1, :]  # Use the last time step
        x = x.view(batch_size, self.num_securities, -1)  # (batch_size, num_securities, num_features_per_security)
        node_embeddings = []
        for i in range(self.num_securities):
            h = self.gcn_layers[i](x[:, i, :])  # (batch_size, 64)
            node_embeddings.append(h)
        h = torch.stack(node_embeddings, dim=1)  # (batch_size, num_securities, 64)
        # Apply attention mechanism
        attn_output, _ = self.attention(h, h, h)
        out = self.fc(attn_output)  # (batch_size, num_securities, num_classes)
        return out

class MambaModel(nn.Module):
    """
    Mamba Model: Combines CNN and LSTM architectures.

    Advantages:
    - Captures both local and long-term dependencies.
    - Utilizes CNN for feature extraction and LSTM for temporal patterns.

    Disadvantages:
    - More complex and computationally intensive.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(MambaModel, self).__init__()
        # Convolutional layer
        self.cnn = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        # LSTM layer
        self.lstm = nn.LSTM(input_size=64, hidden_size=128, num_layers=2, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        # x shape: (batch_size, window_size, num_features)
        x = x.permute(0, 2, 1)  # (batch_size, num_features, window_size)
        x = F.relu(self.cnn(x))  # Apply CNN
        x = x.permute(0, 2, 1)  # (batch_size, new_window_size, channels)
        h0 = torch.zeros(2, x.size(0), 128).to(x.device)  # Initial hidden state
        c0 = torch.zeros(2, x.size(0), 128).to(x.device)  # Initial cell state
        out, _ = self.lstm(x, (h0, c0))  # Apply LSTM
        out = out[:, -1, :]  # Get the last output
        out = self.fc(out)
        out = out.view(-1, self.num_securities, num_classes)
        return out

class LiquidNetModel(nn.Module):
    """
    Liquid Neural Network Model.

    Advantages:
    - Dynamic adaptation to inputs.
    - Good for time-series data.

    Disadvantages:
    - Relatively new, less tested.
    - May be complex to tune.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_securities, num_classes):
        super(LiquidNetModel, self).__init__()
        # RNN Cell with ReLU activation
        self.rnn_cell = nn.RNNCell(input_size, hidden_size, nonlinearity='relu')
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h_t = torch.zeros(x.size(0), self.rnn_cell.hidden_size).to(x.device)  # Initial hidden state
        for t in range(x.size(1)):
            h_t = self.rnn_cell(x[:, t, :], h_t)  # Update hidden state
        out = self.fc(h_t)
        out = out.view(-1, self.num_securities, num_classes)
        return out

class HiddenMarkovModel(nn.Module):
    """
    Hidden Markov Model.

    Advantages:
    - Probabilistic approach.
    - Good for sequential data.

    Disadvantages:
    - Simplified version here.
    - May not capture complex patterns.

    Parameters:
    - num_states (int): Number of hidden states.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_states, num_securities, num_classes):
        super(HiddenMarkovModel, self).__init__()
        self.num_states = num_states
        # Start probabilities
        self.start_prob = nn.Parameter(torch.randn(num_states))
        # Transition probabilities
        self.transition_prob = nn.Parameter(torch.randn(num_states, num_states))
        # Emission probabilities
        self.emission_prob = nn.Parameter(torch.randn(num_states, num_classes))
    def forward(self, x):
        batch_size = x.size(0)
        # Simplified; in practice, you'd implement the forward algorithm
        out = torch.softmax(self.emission_prob, dim=1)
        out = out.unsqueeze(0).unsqueeze(0).repeat(batch_size, self.num_securities, 1)
        return out

class CombinedModel(nn.Module):
    """
    Combined Model: Merges outputs from two models.

    Advantages:
    - Leverages multiple data sources.
    - Potentially better performance.

    Disadvantages:
    - More complex.
    - Computationally intensive.

    Parameters:
    - model1 (nn.Module): First model.
    - model2 (nn.Module): Second model.
    - num_classes (int): Number of target classes.
    - num_securities (int): Number of securities.
    """
    def __init__(self, model1, model2, num_classes, num_securities):
        super(CombinedModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.fc = nn.Linear(num_classes * 2, num_classes)
        self.num_securities = num_securities
        self.num_classes = num_classes
    def forward(self, x1, x2):
        outputs1 = self.model1(x1)
        outputs2 = self.model2(x2)
        outputs = torch.cat((outputs1, outputs2), dim=2)
        batch_size = outputs.size(0)
        outputs = outputs.view(-1, outputs.size(2))
        final_output = self.fc(outputs)
        final_output = final_output.view(batch_size, self.num_securities, -1)
        return final_output

def train_model(model, train_loader, criterion, optimizer, num_epochs, device, writer):
    """
    Training loop for the model.

    Parameters:
    - model (nn.Module): The model to train.
    - train_loader (DataLoader): DataLoader for training data.
    - criterion (nn.Module): Loss function.
    - optimizer (torch.optim.Optimizer): Optimizer.
    - num_epochs (int): Number of epochs.
    - device (torch.device): Computation device.
    - writer (SummaryWriter): TensorBoard SummaryWriter.
    """
    model = model.to(device)
    global_step = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X1_batch, X2_batch, y_batch in train_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X1_batch, X2_batch)
            loss = 0
            for i in range(num_securities):
                loss += criterion(outputs[:, i, :], y_batch[:, i])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            writer.add_scalar('Training Loss', loss.item(), global_step)
            global_step += 1
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
        # Save checkpoint
        torch.save(model.state_dict(), f'checkpoint_epoch_{epoch+1}.pth')

def evaluate_model(model, test_loader, device):
    """
    Evaluate the model on the test set.

    Parameters:
    - model (nn.Module): The model to evaluate.
    - test_loader (DataLoader): DataLoader for test data.
    - device (torch.device): Computation device.

    Returns:
    - float: Cohen's kappa score.
    """
    model = model.to(device)
    model.eval()
    total = 0
    correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X1_batch, X2_batch, y_batch in test_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X1_batch, X2_batch)
            _, predicted = torch.max(outputs.data, 2)
            total += y_batch.numel()
            correct += (predicted == y_batch).sum().item()
            all_preds.extend(predicted.cpu().numpy().flatten())
            all_targets.extend(y_batch.cpu().numpy().flatten())
    accuracy = 100 * correct / total
    kappa = cohen_kappa_score(all_targets, all_preds)
    print(f'Accuracy on test set: {accuracy:.2f}%')
    print(f"Cohen's Kappa: {kappa:.4f}")
    return kappa

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Examples of combination models using the new models

# Example 1: Combine GNNModel and TransformerModel
print("\nTraining Combined Model 1: GNNModel + TransformerModel")
model1 = GNNModel(num_securities=num_securities, num_features_per_security=num_features_per_security, num_classes=num_classes)
model2 = TransformerModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 2: Combine MambaModel and LSTMModel
print("\nTraining Combined Model 2: MambaModel + LSTMModel")
model1 = MambaModel(num_features=num_features, num_securities=num_securities, num_classes=num_classes)
model2 = LSTMModel(input_size=second_X_data.shape[1], hidden_size=64, num_layers=1, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 3: Combine LiquidNetModel and CNNModel
print("\nTraining Combined Model 3: LiquidNetModel + CNNModel")
model1 = LiquidNetModel(input_size=num_features, hidden_size=128, num_securities=num_securities, num_classes=num_classes)
model2 = CNNModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 4: Combine HiddenMarkovModel and LogisticRegressionModel
print("\nTraining Combined Model 4: HiddenMarkovModel + LogisticRegressionModel")
model1 = HiddenMarkovModel(num_states=5, num_securities=num_securities, num_classes=num_classes)
model2 = LogisticRegressionModel(input_size=second_X_data.shape[1]*window_size, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 5: Combine GNNModel and MambaModel
print("\nTraining Combined Model 5: GNNModel + MambaModel")
model1 = GNNModel(num_securities=num_securities, num_features_per_security=num_features_per_security, num_classes=num_classes)
model2 = MambaModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()


Training Combined Model 1: GNNModel + TransformerModel


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: 11.0695
Epoch [2/5], Loss: 11.0211
Epoch [3/5], Loss: 11.0052
Epoch [4/5], Loss: 10.9866
Epoch [5/5], Loss: 10.9811
Accuracy on test set: 32.63%
Cohen's Kappa: -0.0071

Training Combined Model 2: MambaModel + LSTMModel


AttributeError: 'MambaModel' object has no attribute 'num_securities'

In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import cohen_kappa_score
from torch.utils.tensorboard import SummaryWriter
from itertools import product
import time

# Parameters
num_timesteps = 5000  # Reduced for practical purposes
num_securities = 10  # Number of different securities
num_features_per_security = 4  # Features per security
num_classes = 3  # Number of target classes
num_features = num_securities * num_features_per_security  # Total number of features

# Generate timestamps
timestamps = np.arange(num_timesteps)

# Generate random data for the primary dataset (X_data) and labels (y_data)
X_data = np.random.rand(num_timesteps, num_features).astype(np.float32)  # Random features
y_data = np.random.randint(0, num_classes, size=(num_timesteps, num_securities)).astype(np.int64)  # Random labels

# Generate the second dataset with 3 categorical strings and 2 continuous floats per timestamp
str_columns = ['str1', 'str2', 'str3']
float_columns = ['float1', 'float2']
second_dataset = {
    'timestamp': timestamps,
    'str1': np.random.choice(['A', 'B', 'C'], num_timesteps),
    'str2': np.random.choice(['D', 'E', 'F'], num_timesteps),
    'str3': np.random.choice(['G', 'H', 'I'], num_timesteps),
    'float1': np.random.rand(num_timesteps),
    'float2': np.random.rand(num_timesteps),
}

# Encode categorical string columns using LabelEncoder
label_encoders = {}
for col in str_columns:
    le = LabelEncoder()
    second_dataset[col] = le.fit_transform(second_dataset[col])
    label_encoders[col] = le

# Combine all features from the second dataset into a single array
second_X_data = np.column_stack([second_dataset[col] for col in str_columns + float_columns]).astype(np.float32)

# Apply Min-Max scaling to both datasets
scaler_X = MinMaxScaler()
X_data = scaler_X.fit_transform(X_data)

scaler_second_X = MinMaxScaler()
second_X_data = scaler_second_X.fit_transform(second_X_data)

def add_positional_encoding(X, timestamps, option='shared'):
    """
    Add positional encoding to the feature matrix.

    Parameters:
    - X (ndarray): Feature matrix.
    - timestamps (ndarray): Array of timestamps.
    - option (str): 'shared' or 'per_security'.

    Returns:
    - X_pe (ndarray): X with positional encoding added.
    """
    if option == 'shared':
        # Shared positional encoding across all features
        pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(X.shape[1]) / X.shape[1]))
        X_pe = X + pe.astype(np.float32)
    elif option == 'per_security':
        # Separate positional encoding for each security
        pe_list = []
        for i in range(num_securities):
            pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(num_features_per_security) / num_features_per_security))
            pe_list.append(pe)
        pe_concat = np.hstack(pe_list)
        X_pe = X + pe_concat.astype(np.float32)
    else:
        X_pe = X  # No positional encoding
    return X_pe

# Apply positional encoding
positional_encoding_option = 'shared'  # 'shared' or 'per_security'
X_data = add_positional_encoding(X_data, timestamps, positional_encoding_option)

def create_windows(X1, X2, y, window_size, horizon):
    """
    Create sliding windows for time series data.

    Parameters:
    - X1 (ndarray): Primary dataset features.
    - X2 (ndarray): Secondary dataset features.
    - y (ndarray): Target variable.
    - window_size (int): Size of the window.
    - horizon (int): Prediction horizon.

    Returns:
    - Tuple[np.ndarray, np.ndarray, np.ndarray]: Tuples of windows for X1, X2, and y.
    """
    X1_windows = []
    X2_windows = []
    y_windows = []
    for i in range(len(X1) - window_size - horizon + 1):
        X1_windows.append(X1[i:i+window_size])
        X2_windows.append(X2[i:i+window_size])
        y_windows.append(y[i+window_size+horizon-1])
    return np.array(X1_windows), np.array(X2_windows), np.array(y_windows)

# Define window sizes
window_size = 20
horizon = 1

# Create windows
X1_windows, X2_windows, y_windows = create_windows(X_data, second_X_data, y_data, window_size, horizon)

# Split into train and test sets
train_ratio = 0.8
train_size = int(len(X1_windows) * train_ratio)

X1_train = X1_windows[:train_size]
X2_train = X2_windows[:train_size]
y_train = y_windows[:train_size]

X1_test = X1_windows[train_size:]
X2_test = X2_windows[train_size:]
y_test = y_windows[train_size:]

# Convert to PyTorch tensors
X1_train_tensor = torch.tensor(X1_train)
X2_train_tensor = torch.tensor(X2_train)
y_train_tensor = torch.tensor(y_train)

X1_test_tensor = torch.tensor(X1_test)
X2_test_tensor = torch.tensor(X2_test)
y_test_tensor = torch.tensor(y_test)

class TimeSeriesDataset(Dataset):
    """
    Custom Dataset for time series data with two feature sets.

    Parameters:
    - X1 (torch.Tensor): Primary dataset features.
    - X2 (torch.Tensor): Secondary dataset features.
    - y (torch.Tensor): Target variable.
    """
    def __init__(self, X1, X2, y):
        self.X1 = X1.float()
        self.X2 = X2.float()
        self.y = y.long()
    def __len__(self):
        return len(self.X1)
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]

# DataLoaders
batch_size = 64

train_dataset = TimeSeriesDataset(X1_train_tensor, X2_train_tensor, y_train_tensor)
test_dataset = TimeSeriesDataset(X1_test_tensor, X2_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class MatrixRegressionModel(nn.Module):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.

    Parameters:
    - num_securities (int): Number of securities.
    - num_features (int): Total number of features.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_securities, num_features, num_classes):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(1, num_securities))
        self.B = nn.Parameter(torch.randn(num_features, num_classes))
        self.num_securities = num_securities
        self.num_classes = num_classes
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten over time
        out = self.A @ x @ self.B  # Shape: (batch_size, num_classes)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class LogisticRegressionModel(nn.Module):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.

    Parameters:
    - input_size (int): Size of input features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, num_securities, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.linear = nn.Linear(input_size, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.linear(x)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class CNNModel(nn.Module):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(CNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch_size, num_features, window_size)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        out = x.view(-1, self.num_securities, self.num_classes)
        return out

class LSTMModel(nn.Module):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_layers (int): Number of LSTM layers.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_layers, num_securities, num_classes):
        super(LSTMModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class TransformerModel(nn.Module):
    """
    Transformer Model.

    Advantages:
    - Captures global dependencies.
    - Parallel computation.

    Disadvantages:
    - Requires large datasets.
    - Computationally intensive.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(TransformerModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.embedding = nn.Linear(num_features, 128)
        self.pos_encoder = PositionalEncoding(128)
        encoder_layers = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=2)
        self.decoder = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), x.size(1), -1)  # (batch_size, window_size, num_features)
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embedding_dim)
        output = self.transformer_encoder(x)
        output = output[-1, :, :]  # Last output
        output = self.decoder(output)
        output = output.view(-1, self.num_securities, self.num_classes)
        return output

class PositionalEncoding(nn.Module):
    """
    Positional Encoding for Transformer.

    Adds positional information to the embeddings.

    Parameters:
    - d_model (int): Embedding dimension.
    - max_len (int): Maximum sequence length.
    """
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.sin(position * div_term)
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)
    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return x

class SimpleRNNModel(nn.Module):
    """
    Simple Recurrent Neural Network Model.

    Advantages:
    - Captures sequential dependencies.
    - Simpler than LSTM.

    Disadvantages:
    - Struggles with long-term dependencies.
    - May suffer from vanishing gradients.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_securities, num_classes):
        super(SimpleRNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

# Missing Models with Docstrings and Comments

class GNNModel(nn.Module):
    """
    Graph Neural Network Model.

    Advantages:
    - Captures relationships between securities.
    - Utilizes attention mechanisms.

    Disadvantages:
    - Computationally intensive.
    - Requires graph structure.

    Parameters:
    - num_securities (int): Number of securities.
    - num_features_per_security (int): Features per security.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_securities, num_features_per_security, num_classes):
        super(GNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        # Create a linear layer for each security
        self.gcn_layers = nn.ModuleList([nn.Linear(num_features_per_security, 64) for _ in range(num_securities)])
        # Multi-head attention mechanism
        self.attention = nn.MultiheadAttention(embed_dim=64, num_heads=4, batch_first=True)
        # Fully connected layer for classification
        self.fc = nn.Linear(64, num_classes)
    def forward(self, x):
        # x shape: (batch_size, window_size, num_features)
        batch_size = x.size(0)
        x = x[:, -1, :]  # Use the last time step
        x = x.view(batch_size, self.num_securities, -1)  # (batch_size, num_securities, num_features_per_security)
        node_embeddings = []
        for i in range(self.num_securities):
            h = self.gcn_layers[i](x[:, i, :])  # (batch_size, 64)
            node_embeddings.append(h)
        h = torch.stack(node_embeddings, dim=1)  # (batch_size, num_securities, 64)
        # Apply attention mechanism
        attn_output, _ = self.attention(h, h, h)
        out = self.fc(attn_output)  # (batch_size, num_securities, num_classes)
        return out

class MambaModel(nn.Module):
    """
    Mamba Model: Combines CNN and LSTM architectures.

    Advantages:
    - Captures both local and long-term dependencies.
    - Utilizes CNN for feature extraction and LSTM for temporal patterns.

    Disadvantages:
    - More complex and computationally intensive.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(MambaModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        # Convolutional layer
        self.cnn = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        # LSTM layer
        self.lstm = nn.LSTM(input_size=64, hidden_size=128, num_layers=2, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        # x shape: (batch_size, window_size, num_features)
        x = x.permute(0, 2, 1)  # (batch_size, num_features, window_size)
        x = F.relu(self.cnn(x))  # Apply CNN
        x = x.permute(0, 2, 1)  # (batch_size, new_window_size, channels)
        h0 = torch.zeros(2, x.size(0), 128).to(x.device)  # Initial hidden state
        c0 = torch.zeros(2, x.size(0), 128).to(x.device)  # Initial cell state
        out, _ = self.lstm(x, (h0, c0))  # Apply LSTM
        out = out[:, -1, :]  # Get the last output
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class LiquidNetModel(nn.Module):
    """
    Liquid Neural Network Model.

    Advantages:
    - Dynamic adaptation to inputs.
    - Good for time-series data.

    Disadvantages:
    - Relatively new, less tested.
    - May be complex to tune.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_securities, num_classes):
        super(LiquidNetModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        # RNN Cell with ReLU activation
        self.rnn_cell = nn.RNNCell(input_size, hidden_size, nonlinearity='relu')
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h_t = torch.zeros(x.size(0), self.rnn_cell.hidden_size).to(x.device)  # Initial hidden state
        for t in range(x.size(1)):
            h_t = self.rnn_cell(x[:, t, :], h_t)  # Update hidden state
        out = self.fc(h_t)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class HiddenMarkovModel(nn.Module):
    """
    Hidden Markov Model.

    Advantages:
    - Probabilistic approach.
    - Good for sequential data.

    Disadvantages:
    - Simplified version here.
    - May not capture complex patterns.

    Parameters:
    - num_states (int): Number of hidden states.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_states, num_securities, num_classes):
        super(HiddenMarkovModel, self).__init__()
        self.num_states = num_states
        self.num_securities = num_securities
        self.num_classes = num_classes
        # Start probabilities
        self.start_prob = nn.Parameter(torch.randn(num_states))
        # Transition probabilities
        self.transition_prob = nn.Parameter(torch.randn(num_states, num_states))
        # Emission probabilities
        self.emission_prob = nn.Parameter(torch.randn(num_states, num_classes))
    def forward(self, x):
        batch_size = x.size(0)
        # Simplified; in practice, you'd implement the forward algorithm
        out = torch.softmax(self.emission_prob, dim=1)
        out = out.unsqueeze(0).unsqueeze(0).repeat(batch_size, self.num_securities, 1)
        return out

class CombinedModel(nn.Module):
    """
    Combined Model: Merges outputs from two models.

    Advantages:
    - Leverages multiple data sources.
    - Potentially better performance.

    Disadvantages:
    - More complex.
    - Computationally intensive.

    Parameters:
    - model1 (nn.Module): First model.
    - model2 (nn.Module): Second model.
    - num_classes (int): Number of target classes.
    - num_securities (int): Number of securities.
    """
    def __init__(self, model1, model2, num_classes, num_securities):
        super(CombinedModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.fc = nn.Linear(num_classes * 2, num_classes)
        self.num_securities = num_securities
        self.num_classes = num_classes
    def forward(self, x1, x2):
        outputs1 = self.model1(x1)
        outputs2 = self.model2(x2)
        outputs = torch.cat((outputs1, outputs2), dim=2)
        batch_size = outputs.size(0)
        outputs = outputs.view(-1, outputs.size(2))
        final_output = self.fc(outputs)
        final_output = final_output.view(batch_size, self.num_securities, -1)
        return final_output

def train_model(model, train_loader, criterion, optimizer, num_epochs, device, writer):
    """
    Training loop for the model.

    Parameters:
    - model (nn.Module): The model to train.
    - train_loader (DataLoader): DataLoader for training data.
    - criterion (nn.Module): Loss function.
    - optimizer (torch.optim.Optimizer): Optimizer.
    - num_epochs (int): Number of epochs.
    - device (torch.device): Computation device.
    - writer (SummaryWriter): TensorBoard SummaryWriter.
    """
    model = model.to(device)
    global_step = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X1_batch, X2_batch, y_batch in train_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X1_batch, X2_batch)
            loss = 0
            for i in range(num_securities):
                loss += criterion(outputs[:, i, :], y_batch[:, i])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            writer.add_scalar('Training Loss', loss.item(), global_step)
            global_step += 1
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
        # Save checkpoint
        torch.save(model.state_dict(), f'checkpoint_epoch_{epoch+1}.pth')

def evaluate_model(model, test_loader, device):
    """
    Evaluate the model on the test set.

    Parameters:
    - model (nn.Module): The model to evaluate.
    - test_loader (DataLoader): DataLoader for test data.
    - device (torch.device): Computation device.

    Returns:
    - float: Cohen's kappa score.
    """
    model = model.to(device)
    model.eval()
    total = 0
    correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X1_batch, X2_batch, y_batch in test_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X1_batch, X2_batch)
            _, predicted = torch.max(outputs.data, 2)
            total += y_batch.numel()
            correct += (predicted == y_batch).sum().item()
            all_preds.extend(predicted.cpu().numpy().flatten())
            all_targets.extend(y_batch.cpu().numpy().flatten())
    accuracy = 100 * correct / total
    kappa = cohen_kappa_score(all_targets, all_preds)
    print(f'Accuracy on test set: {accuracy:.2f}%')
    print(f"Cohen's Kappa: {kappa:.4f}")
    return kappa

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Examples of combination models using the new models

# Example 1: Combine GNNModel and TransformerModel
print("\nTraining Combined Model 1: GNNModel + TransformerModel")
model1 = GNNModel(num_securities=num_securities, num_features_per_security=num_features_per_security, num_classes=num_classes)
model2 = TransformerModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 2: Combine MambaModel and LSTMModel
print("\nTraining Combined Model 2: MambaModel + LSTMModel")
model1 = MambaModel(num_features=num_features, num_securities=num_securities, num_classes=num_classes)
model2 = LSTMModel(input_size=second_X_data.shape[1], hidden_size=64, num_layers=1, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 3: Combine LiquidNetModel and CNNModel
print("\nTraining Combined Model 3: LiquidNetModel + CNNModel")
model1 = LiquidNetModel(input_size=num_features, hidden_size=128, num_securities=num_securities, num_classes=num_classes)
model2 = CNNModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 4: Combine HiddenMarkovModel and LogisticRegressionModel
print("\nTraining Combined Model 4: HiddenMarkovModel + LogisticRegressionModel")
model1 = HiddenMarkovModel(num_states=5, num_securities=num_securities, num_classes=num_classes)
model2 = LogisticRegressionModel(input_size=second_X_data.shape[1]*window_size, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 5: Combine GNNModel and MambaModel
print("\nTraining Combined Model 5: GNNModel + MambaModel")
model1 = GNNModel(num_securities=num_securities, num_features_per_security=num_features_per_security, num_classes=num_classes)
model2 = MambaModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()


Training Combined Model 1: GNNModel + TransformerModel


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: 11.0633
Epoch [2/5], Loss: 11.0170
Epoch [3/5], Loss: 11.0033
Epoch [4/5], Loss: 10.9927
Epoch [5/5], Loss: 10.9869
Accuracy on test set: 33.32%
Cohen's Kappa: -0.0002

Training Combined Model 2: MambaModel + LSTMModel
Epoch [1/5], Loss: 10.9977
Epoch [2/5], Loss: 10.9867
Epoch [3/5], Loss: 10.9827
Epoch [4/5], Loss: 10.9759
Epoch [5/5], Loss: 10.9718
Accuracy on test set: 32.71%
Cohen's Kappa: -0.0092

Training Combined Model 3: LiquidNetModel + CNNModel
Epoch [1/5], Loss: 11.0257
Epoch [2/5], Loss: 10.9812
Epoch [3/5], Loss: 10.9605
Epoch [4/5], Loss: 10.9400
Epoch [5/5], Loss: 10.9039
Accuracy on test set: 33.71%
Cohen's Kappa: 0.0057

Training Combined Model 4: HiddenMarkovModel + LogisticRegressionModel


RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor

In [22]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import cohen_kappa_score
from torch.utils.tensorboard import SummaryWriter
from itertools import product
import time

# Parameters
num_timesteps = 5000  # Total number of time steps
num_securities = 10  # Number of different securities
num_features_per_security = 4  # Features per security
num_classes = 3  # Number of target classes
num_features = num_securities * num_features_per_security  # Total number of features

# Generate timestamps
timestamps = np.arange(num_timesteps)

# Generate random data for the primary dataset (X_data) and labels (y_data)
X_data = np.random.rand(num_timesteps, num_features).astype(np.float32)  # Random features
y_data = np.random.randint(0, num_classes, size=(num_timesteps, num_securities)).astype(np.int64)  # Random labels

# Generate the second dataset with 3 categorical strings and 2 continuous floats per timestamp
str_columns = ['str1', 'str2', 'str3']
float_columns = ['float1', 'float2']
second_dataset = {
    'timestamp': timestamps,
    'str1': np.random.choice(['A', 'B', 'C'], num_timesteps),
    'str2': np.random.choice(['D', 'E', 'F'], num_timesteps),
    'str3': np.random.choice(['G', 'H', 'I'], num_timesteps),
    'float1': np.random.rand(num_timesteps),
    'float2': np.random.rand(num_timesteps),
}

# Encode categorical string columns using LabelEncoder
label_encoders = {}
for col in str_columns:
    le = LabelEncoder()
    second_dataset[col] = le.fit_transform(second_dataset[col])
    label_encoders[col] = le

# Combine all features from the second dataset into a single array
second_X_data = np.column_stack([second_dataset[col] for col in str_columns + float_columns]).astype(np.float32)

# Apply Min-Max scaling to both datasets
scaler_X = MinMaxScaler()
X_data = scaler_X.fit_transform(X_data)

scaler_second_X = MinMaxScaler()
second_X_data = scaler_second_X.fit_transform(second_X_data)

def add_positional_encoding(X, timestamps, option='shared'):
    """
    Add positional encoding to the feature matrix.

    Parameters:
    - X (ndarray): Feature matrix.
    - timestamps (ndarray): Array of timestamps.
    - option (str): 'shared' or 'per_security'.

    Returns:
    - X_pe (ndarray): X with positional encoding added.
    """
    if option == 'shared':
        # Shared positional encoding across all features
        pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(X.shape[1]) / X.shape[1]))
        X_pe = X + pe.astype(np.float32)
    elif option == 'per_security':
        # Separate positional encoding for each security
        pe_list = []
        for i in range(num_securities):
            pe = np.sin(timestamps[:, None] / 10000 ** (np.arange(num_features_per_security) / num_features_per_security))
            pe_list.append(pe)
        pe_concat = np.hstack(pe_list)
        X_pe = X + pe_concat.astype(np.float32)
    else:
        X_pe = X  # No positional encoding
    return X_pe

# Apply positional encoding
positional_encoding_option = 'shared'  # 'shared' or 'per_security'
X_data = add_positional_encoding(X_data, timestamps, positional_encoding_option)

def create_windows(X1, X2, y, window_size, horizon):
    """
    Create sliding windows for time series data.

    Parameters:
    - X1 (ndarray): Primary dataset features.
    - X2 (ndarray): Secondary dataset features.
    - y (ndarray): Target variable.
    - window_size (int): Size of the window.
    - horizon (int): Prediction horizon.

    Returns:
    - Tuple[np.ndarray, np.ndarray, np.ndarray]: Tuples of windows for X1, X2, and y.
    """
    X1_windows = []
    X2_windows = []
    y_windows = []
    for i in range(len(X1) - window_size - horizon + 1):
        X1_windows.append(X1[i:i+window_size])
        X2_windows.append(X2[i:i+window_size])
        y_windows.append(y[i+window_size+horizon-1])
    return np.array(X1_windows), np.array(X2_windows), np.array(y_windows)

# Define window sizes
window_size = 20
horizon = 1

# Create windows
X1_windows, X2_windows, y_windows = create_windows(X_data, second_X_data, y_data, window_size, horizon)

# Split into train and test sets
train_ratio = 0.8
train_size = int(len(X1_windows) * train_ratio)

X1_train = X1_windows[:train_size]
X2_train = X2_windows[:train_size]
y_train = y_windows[:train_size]

X1_test = X1_windows[train_size:]
X2_test = X2_windows[train_size:]
y_test = y_windows[train_size:]

# Convert to PyTorch tensors
X1_train_tensor = torch.tensor(X1_train)
X2_train_tensor = torch.tensor(X2_train)
y_train_tensor = torch.tensor(y_train)

X1_test_tensor = torch.tensor(X1_test)
X2_test_tensor = torch.tensor(X2_test)
y_test_tensor = torch.tensor(y_test)

class TimeSeriesDataset(Dataset):
    """
    Custom Dataset for time series data with two feature sets.

    Parameters:
    - X1 (torch.Tensor): Primary dataset features.
    - X2 (torch.Tensor): Secondary dataset features.
    - y (torch.Tensor): Target variable.
    """
    def __init__(self, X1, X2, y):
        self.X1 = X1.float()
        self.X2 = X2.float()
        self.y = y.long()
    def __len__(self):
        return len(self.X1)
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]

# DataLoaders
batch_size = 64

train_dataset = TimeSeriesDataset(X1_train_tensor, X2_train_tensor, y_train_tensor)
test_dataset = TimeSeriesDataset(X1_test_tensor, X2_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class MatrixRegressionModel(nn.Module):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.

    Parameters:
    - num_securities (int): Number of securities.
    - num_features (int): Total number of features.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_securities, num_features, num_classes):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(1, num_securities))
        self.B = nn.Parameter(torch.randn(num_features, num_classes))
        self.num_securities = num_securities
        self.num_classes = num_classes
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten over time
        out = self.A @ x @ self.B  # Shape: (batch_size, num_classes)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class LogisticRegressionModel(nn.Module):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.

    Parameters:
    - input_size (int): Size of input features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, num_securities, num_classes):
        super(LogisticRegressionModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.linear = nn.Linear(input_size, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.linear(x)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class CNNModel(nn.Module):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(CNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch_size, num_features, window_size)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        out = x.view(-1, self.num_securities, self.num_classes)
        return out

class LSTMModel(nn.Module):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_layers (int): Number of LSTM layers.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_layers, num_securities, num_classes):
        super(LSTMModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class TransformerModel(nn.Module):
    """
    Transformer Model.

    Advantages:
    - Captures global dependencies.
    - Parallel computation.

    Disadvantages:
    - Requires large datasets.
    - Computationally intensive.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(TransformerModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.embedding = nn.Linear(num_features, 128)
        self.pos_encoder = PositionalEncoding(128)
        encoder_layers = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=2)
        self.decoder = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        x = x.view(x.size(0), x.size(1), -1)  # (batch_size, window_size, num_features)
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embedding_dim)
        output = self.transformer_encoder(x)
        output = output[-1, :, :]  # Last output
        output = self.decoder(output)
        output = output.view(-1, self.num_securities, self.num_classes)
        return output

class PositionalEncoding(nn.Module):
    """
    Positional Encoding for Transformer.

    Adds positional information to the embeddings.

    Parameters:
    - d_model (int): Embedding dimension.
    - max_len (int): Maximum sequence length.
    """
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.sin(position * div_term)
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)
    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return x

class SimpleRNNModel(nn.Module):
    """
    Simple Recurrent Neural Network Model.

    Advantages:
    - Captures sequential dependencies.
    - Simpler than LSTM.

    Disadvantages:
    - Struggles with long-term dependencies.
    - May suffer from vanishing gradients.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_securities, num_classes):
        super(SimpleRNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

# Missing Models with Docstrings and Comments

class GNNModel(nn.Module):
    """
    Graph Neural Network Model.

    Advantages:
    - Captures relationships between securities.
    - Utilizes attention mechanisms.

    Disadvantages:
    - Computationally intensive.
    - Requires graph structure.

    Parameters:
    - num_securities (int): Number of securities.
    - num_features_per_security (int): Features per security.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_securities, num_features_per_security, num_classes):
        super(GNNModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        # Create a linear layer for each security
        self.gcn_layers = nn.ModuleList([nn.Linear(num_features_per_security, 64) for _ in range(num_securities)])
        # Multi-head attention mechanism
        self.attention = nn.MultiheadAttention(embed_dim=64, num_heads=4, batch_first=True)
        # Fully connected layer for classification
        self.fc = nn.Linear(64, num_classes)
    def forward(self, x):
        # x shape: (batch_size, window_size, num_features)
        batch_size = x.size(0)
        x = x[:, -1, :]  # Use the last time step
        x = x.view(batch_size, self.num_securities, -1)  # (batch_size, num_securities, num_features_per_security)
        node_embeddings = []
        for i in range(self.num_securities):
            h = self.gcn_layers[i](x[:, i, :])  # (batch_size, 64)
            node_embeddings.append(h)
        h = torch.stack(node_embeddings, dim=1)  # (batch_size, num_securities, 64)
        # Apply attention mechanism
        attn_output, _ = self.attention(h, h, h)
        out = self.fc(attn_output)  # (batch_size, num_securities, num_classes)
        return out

class MambaModel(nn.Module):
    """
    Mamba Model: Combines CNN and LSTM architectures.

    Advantages:
    - Captures both local and long-term dependencies.
    - Utilizes CNN for feature extraction and LSTM for temporal patterns.

    Disadvantages:
    - More complex and computationally intensive.

    Parameters:
    - num_features (int): Total number of features.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_features, num_securities, num_classes):
        super(MambaModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        # Convolutional layer
        self.cnn = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        # LSTM layer
        self.lstm = nn.LSTM(input_size=64, hidden_size=128, num_layers=2, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(128, num_securities * num_classes)
    def forward(self, x):
        # x shape: (batch_size, window_size, num_features)
        x = x.permute(0, 2, 1)  # (batch_size, num_features, window_size)
        x = F.relu(self.cnn(x))  # Apply CNN
        x = x.permute(0, 2, 1)  # (batch_size, new_window_size, channels)
        h0 = torch.zeros(2, x.size(0), 128).to(x.device)  # Initial hidden state
        c0 = torch.zeros(2, x.size(0), 128).to(x.device)  # Initial cell state
        out, _ = self.lstm(x, (h0, c0))  # Apply LSTM
        out = out[:, -1, :]  # Get the last output
        out = self.fc(out)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class LiquidNetModel(nn.Module):
    """
    Liquid Neural Network Model.

    Advantages:
    - Dynamic adaptation to inputs.
    - Good for time-series data.

    Disadvantages:
    - Relatively new, less tested.
    - May be complex to tune.

    Parameters:
    - input_size (int): Size of input features.
    - hidden_size (int): Size of hidden state.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, input_size, hidden_size, num_securities, num_classes):
        super(LiquidNetModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        # RNN Cell with ReLU activation
        self.rnn_cell = nn.RNNCell(input_size, hidden_size, nonlinearity='relu')
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)
    def forward(self, x):
        h_t = torch.zeros(x.size(0), self.rnn_cell.hidden_size).to(x.device)  # Initial hidden state
        for t in range(x.size(1)):
            h_t = self.rnn_cell(x[:, t, :], h_t)  # Update hidden state
        out = self.fc(h_t)
        out = out.view(-1, self.num_securities, self.num_classes)
        return out

class HiddenMarkovModel(nn.Module):
    """
    Hidden Markov Model.

    Advantages:
    - Probabilistic approach.
    - Good for sequential data.

    Disadvantages:
    - Simplified version here.
    - May not capture complex patterns.

    Parameters:
    - num_states (int): Number of hidden states.
    - num_securities (int): Number of securities.
    - num_classes (int): Number of target classes.
    """
    def __init__(self, num_states, num_securities, num_classes):
        super(HiddenMarkovModel, self).__init__()
        self.num_states = num_states
        self.num_securities = num_securities
        self.num_classes = num_classes
        # Start probabilities
        self.start_prob = nn.Parameter(torch.randn(num_states))
        # Transition probabilities
        self.transition_prob = nn.Parameter(torch.randn(num_states, num_states))
        # Emission probabilities
        self.emission_prob = nn.Parameter(torch.randn(num_states, num_classes))
    def forward(self, x):
        batch_size = x.size(0)
        # Simplified; in practice, you'd implement the forward algorithm
        out = torch.softmax(self.emission_prob, dim=1)
        # Corrected repeat dimensions to match tensor dimensions
        out = out.unsqueeze(0).unsqueeze(0).repeat(batch_size, self.num_securities, 1, 1)
        # Since the model expects output shape (batch_size, num_securities, num_classes)
        # We need to aggregate over the hidden states (num_states)
        out = torch.mean(out, dim=2)  # Average over hidden states
        return out

class CombinedModel(nn.Module):
    """
    Combined Model: Merges outputs from two models.

    Advantages:
    - Leverages multiple data sources.
    - Potentially better performance.

    Disadvantages:
    - More complex.
    - Computationally intensive.

    Parameters:
    - model1 (nn.Module): First model.
    - model2 (nn.Module): Second model.
    - num_classes (int): Number of target classes.
    - num_securities (int): Number of securities.
    """
    def __init__(self, model1, model2, num_classes, num_securities):
        super(CombinedModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.fc = nn.Linear(num_classes * 2, num_classes)
        self.num_securities = num_securities
        self.num_classes = num_classes
    def forward(self, x1, x2):
        outputs1 = self.model1(x1)
        outputs2 = self.model2(x2)
        outputs = torch.cat((outputs1, outputs2), dim=2)
        batch_size = outputs.size(0)
        outputs = outputs.view(-1, outputs.size(2))
        final_output = self.fc(outputs)
        final_output = final_output.view(batch_size, self.num_securities, -1)
        return final_output

def train_model(model, train_loader, criterion, optimizer, num_epochs, device, writer):
    """
    Training loop for the model.

    Parameters:
    - model (nn.Module): The model to train.
    - train_loader (DataLoader): DataLoader for training data.
    - criterion (nn.Module): Loss function.
    - optimizer (torch.optim.Optimizer): Optimizer.
    - num_epochs (int): Number of epochs.
    - device (torch.device): Computation device.
    - writer (SummaryWriter): TensorBoard SummaryWriter.
    """
    model = model.to(device)
    global_step = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X1_batch, X2_batch, y_batch in train_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X1_batch, X2_batch)
            loss = 0
            for i in range(num_securities):
                loss += criterion(outputs[:, i, :], y_batch[:, i])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            writer.add_scalar('Training Loss', loss.item(), global_step)
            global_step += 1
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
        # Save checkpoint
        torch.save(model.state_dict(), f'checkpoint_epoch_{epoch+1}.pth')

def evaluate_model(model, test_loader, device):
    """
    Evaluate the model on the test set.

    Parameters:
    - model (nn.Module): The model to evaluate.
    - test_loader (DataLoader): DataLoader for test data.
    - device (torch.device): Computation device.

    Returns:
    - float: Cohen's kappa score.
    """
    model = model.to(device)
    model.eval()
    total = 0
    correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X1_batch, X2_batch, y_batch in test_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X1_batch, X2_batch)
            _, predicted = torch.max(outputs.data, 2)
            total += y_batch.numel()
            correct += (predicted == y_batch).sum().item()
            all_preds.extend(predicted.cpu().numpy().flatten())
            all_targets.extend(y_batch.cpu().numpy().flatten())
    accuracy = 100 * correct / total
    kappa = cohen_kappa_score(all_targets, all_preds)
    print(f'Accuracy on test set: {accuracy:.2f}%')
    print(f"Cohen's Kappa: {kappa:.4f}")
    return kappa

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Examples of combination models using the new models

# Example 1: Combine GNNModel and TransformerModel
print("\nTraining Combined Model 1: GNNModel + TransformerModel")
model1 = GNNModel(num_securities=num_securities, num_features_per_security=num_features_per_security, num_classes=num_classes)
model2 = TransformerModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 2: Combine MambaModel and LSTMModel
print("\nTraining Combined Model 2: MambaModel + LSTMModel")
model1 = MambaModel(num_features=num_features, num_securities=num_securities, num_classes=num_classes)
model2 = LSTMModel(input_size=second_X_data.shape[1], hidden_size=64, num_layers=1, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 3: Combine LiquidNetModel and CNNModel
print("\nTraining Combined Model 3: LiquidNetModel + CNNModel")
model1 = LiquidNetModel(input_size=num_features, hidden_size=128, num_securities=num_securities, num_classes=num_classes)
model2 = CNNModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 4: Combine HiddenMarkovModel and LogisticRegressionModel
print("\nTraining Combined Model 4: HiddenMarkovModel + LogisticRegressionModel")
model1 = HiddenMarkovModel(num_states=5, num_securities=num_securities, num_classes=num_classes)
model2 = LogisticRegressionModel(input_size=second_X_data.shape[1]*window_size, num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()

# Example 5: Combine GNNModel and MambaModel
print("\nTraining Combined Model 5: GNNModel + MambaModel")
model1 = GNNModel(num_securities=num_securities, num_features_per_security=num_features_per_security, num_classes=num_classes)
model2 = MambaModel(num_features=second_X_data.shape[1], num_securities=num_securities, num_classes=num_classes)
combined_model = CombinedModel(model1, model2, num_classes, num_securities)
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
writer = SummaryWriter()
train_model(combined_model, train_loader, criterion, optimizer, num_epochs=5, device=device, writer=writer)
evaluate_model(combined_model, test_loader, device)
writer.close()


Training Combined Model 1: GNNModel + TransformerModel


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: 11.0967
Epoch [2/5], Loss: 11.0186
Epoch [3/5], Loss: 11.0025
Epoch [4/5], Loss: 10.9973
Epoch [5/5], Loss: 10.9875
Accuracy on test set: 33.69%
Cohen's Kappa: 0.0052

Training Combined Model 2: MambaModel + LSTMModel
Epoch [1/5], Loss: 11.0083
Epoch [2/5], Loss: 10.9869
Epoch [3/5], Loss: 10.9811
Epoch [4/5], Loss: 10.9745
Epoch [5/5], Loss: 10.9668
Accuracy on test set: 32.87%
Cohen's Kappa: -0.0069

Training Combined Model 3: LiquidNetModel + CNNModel
Epoch [1/5], Loss: 11.0202
Epoch [2/5], Loss: 10.9822
Epoch [3/5], Loss: 10.9718
Epoch [4/5], Loss: 10.9498
Epoch [5/5], Loss: 10.9361
Accuracy on test set: 33.54%
Cohen's Kappa: 0.0026

Training Combined Model 4: HiddenMarkovModel + LogisticRegressionModel
Epoch [1/5], Loss: 11.0855
Epoch [2/5], Loss: 10.9962
Epoch [3/5], Loss: 10.9837
Epoch [4/5], Loss: 10.9777
Epoch [5/5], Loss: 10.9678
Accuracy on test set: 33.48%
Cohen's Kappa: 0.0018

Training Combined Model 5: GNNModel + MambaModel
Epoch [1/5], Loss: 11.0007
E

In [23]:
# ... [Previous code remains the same up to model definitions]

# Instantiate models for primary and second datasets
print("Training Combined Model with Transformer and LSTM...")

# Primary model
primary_model = TransformerModel(num_features=num_features, num_securities=num_securities, num_classes=num_classes)
# Secondary model
secondary_model = LSTMModel(input_size=second_input_size, hidden_size=128, num_layers=2, num_securities=num_securities, num_classes=num_classes)

# Define final linear layer
class FinalModel(nn.Module):
    def __init__(self, num_classes, num_securities):
        super(FinalModel, self).__init__()
        self.fc = nn.Linear(num_classes * 2, num_classes)
        self.num_securities = num_securities
    def forward(self, outputs1, outputs2):
        # Concatenate over class dimension
        outputs = torch.cat((outputs1, outputs2), dim=2)
        # Pass through linear layer
        batch_size = outputs.size(0)
        outputs = outputs.view(-1, outputs.size(2))
        final_output = self.fc(outputs)
        final_output = final_output.view(batch_size, self.num_securities, -1)
        return final_output

final_model = FinalModel(num_classes=num_classes, num_securities=num_securities)

# Combine the parameters of all models
params = list(primary_model.parameters()) + list(secondary_model.parameters()) + list(final_model.parameters())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Updated training function
def train_model(model1, model2, final_model, train_loader, criterion, optimizer, num_epochs, device):
    model1 = model1.to(device)
    model2 = model2.to(device)
    final_model = final_model.to(device)
    for epoch in range(num_epochs):
        model1.train()
        model2.train()
        final_model.train()
        total_loss = 0
        for X1_batch, X2_batch, y_batch in train_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs1 = model1(X1_batch)
            outputs2 = model2(X2_batch)
            final_output = final_model(outputs1, outputs2)
            loss = 0
            for i in range(num_securities):
                loss += criterion(final_output[:, i, :], y_batch[:, i])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Updated evaluation function
def evaluate_model(model1, model2, final_model, test_loader, device):
    model1 = model1.to(device)
    model2 = model2.to(device)
    final_model = final_model.to(device)
    model1.eval()
    model2.eval()
    final_model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for X1_batch, X2_batch, y_batch in test_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            outputs1 = model1(X1_batch)
            outputs2 = model2(X2_batch)
            final_output = final_model(outputs1, outputs2)
            _, predicted = torch.max(final_output.data, 2)
            total += y_batch.numel()
            correct += (predicted == y_batch).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

# Start training
train_model(primary_model, secondary_model, final_model, train_loader, criterion, optimizer, num_epochs, device)
evaluate_model(primary_model, secondary_model, final_model, test_loader, device)

Training Combined Model with Transformer and LSTM...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: 11.0751
Epoch [2/5], Loss: 11.0117
Epoch [3/5], Loss: 11.0131
Epoch [4/5], Loss: 11.0011
Epoch [5/5], Loss: 10.9982
Accuracy on test set: 32.97%


In [ ]:
!pip install mamba-ssm[dev]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 959.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.4 MB/s eta 0:00:00
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=323988104 sha256=6b082468a6abb6f6bc50c99263f17c6c7f5a2e8f6b275ed7998b81fb25279229
  Stored in directory: /root/.cache/pip/wheels/57/7c/90/9f963468ecc3791e36e388f9e7b4a4e1e3f90fbb340055aa4d
Successfully built mamba-ssm


In [ ]:
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn

class HiddenMarkovModel(nn.Module):
    def __init__(self, num_states, num_securities, num_classes, num_features_per_security):
        super(HiddenMarkovModel, self).__init__()
        self.num_states = num_states
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.num_features_per_security = num_features_per_security

        # Define parameters for start, transition, and emission probabilities
        self.start_logits = nn.Parameter(torch.randn(num_securities, num_states))
        self.trans_logits = nn.Parameter(torch.randn(num_securities, num_states, num_states))
        self.emission_logits = nn.Parameter(torch.randn(num_securities, num_states, num_classes))

    def forward(self, x):
        # x shape: (batch_size, window_size, num_features)
        batch_size, window_size, num_features = x.size()
        device = x.device

        # Reshape x to separate securities
        x = x.view(batch_size, window_size, self.num_securities, self.num_features_per_security)
        # For simplicity, we'll use the last time step
        x_last = x[:, -1, :, :]  # Shape: (batch_size, num_securities, num_features_per_security)

        # We'll compute the logits for each class per security
        # For this simplified HMM, we'll combine start and emission logits to produce class logits
        outputs = []
        for s in range(self.num_securities):
            # Compute the log probabilities for each class
            # We use log-sum-exp over the states to compute the logits for each class
            # start_logits[s]: (num_states,)
            # emission_logits[s]: (num_states, num_classes)
            start_log_probs = self.start_logits[s]  # (num_states,)
            emission_log_probs = self.emission_logits[s]  # (num_states, num_classes)

            # Compute logits for classes
            logits = torch.logsumexp(start_log_probs.unsqueeze(1) + emission_log_probs, dim=0)  # (num_classes,)
            outputs.append(logits.unsqueeze(0))  # (1, num_classes)

        # Stack outputs over securities and expand to batch size
        outputs = torch.cat(outputs, dim=0)  # (num_securities, num_classes)
        outputs = outputs.unsqueeze(0).expand(batch_size, -1, -1)  # (batch_size, num_securities, num_classes)

        return outputs  # Shape: (batch_size, num_securities, num_classes)


# Adjusted training loop for HMM with continuous emissions
def train_hmm_model(model, train_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for _, _, y_batch in train_loader:
            y_batch = y_batch.to(device)  # Shape: (batch_size, num_securities)
            y_batch = y_batch.unsqueeze(1).expand(-1, window_size, -1)  # (batch_size, window_size, num_securities)

            optimizer.zero_grad()
            outputs = model(y_batch)  # outputs shape: (batch_size, window_size, num_securities)
            loss = criterion(outputs.view(-1), torch.zeros_like(outputs.view(-1)))  # Dummy target
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')
# Mamba Model Integration
# Since you will install the Mamba package, we'll import and use it directly
try:
    from mamba_ssm.modules.mamba2 import Mamba2
except ImportError:
    raise ImportError("Please install the mamba_ssm package to use the MambaModel.")

class MambaModel(nn.Module):
    def __init__(self, num_features, num_securities, num_classes):
        super(MambaModel, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.mamba = Mamba2(
            d_model=num_features,
            d_state=64,
            d_conv=4,
            expand=2,
        )

        # Final linear layer to map to the desired output size
        self.fc = nn.Linear(num_features, num_securities * num_classes)

    def forward(self, x):
        # x shape: (batch_size, window_size, num_features)
        batch_size = x.size(0)
        x = x.view(batch_size, -1, x.size(-1))  # Ensure proper shape
        y = self.mamba(x)
        y = self.fc(y[:, -1, :])  # Use the last time step
        y = y.view(batch_size, self.num_securities, self.num_classes)
        return y



In [ ]:
def train_single_model(model, train_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, _, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = 0
            for i in range(num_securities):
                loss += criterion(outputs[:, i, :], y_batch[:, i])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

def evaluate_single_model(model, test_loader, device):
    model = model.to(device)
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for X_batch, _, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 2)
            total += y_batch.numel()
            correct += (predicted == y_batch).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

In [ ]:
# Instantiate and train Hidden Markov Model
print("\nTraining Hidden Markov Model...")
num_states = 5  # Number of hidden states

hmm_model = HiddenMarkovModel(
    num_states=num_states,
    num_securities=num_securities,
    num_classes=num_classes,
    num_features_per_security=num_features//num_securities,
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(hmm_model.parameters(), lr=learning_rate)

train_single_model(hmm_model, train_loader, criterion, optimizer, num_epochs, device)
evaluate_single_model(hmm_model, test_loader, device)


Training Hidden Markov Model...
Epoch [1/5], Loss: 11.9516
Epoch [2/5], Loss: 11.0472
Epoch [3/5], Loss: 10.9873
Epoch [4/5], Loss: 10.9865
Epoch [5/5], Loss: 10.9865
Accuracy on test set: 33.44%


In [ ]:
class LSTMModelData2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_securities, num_classes):
        super(LSTMModelData2, self).__init__()
        self.num_securities = num_securities
        self.num_classes = num_classes
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_securities * num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Use the last time step
        out = out.view(batch_size, self.num_securities, self.num_classes)
        return out

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X1, X2, y):
        self.X1 = X1.float()
        self.X2 = X2.float()
        self.y = y.long()
    def __len__(self):
        return len(self.X1)
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]

# DataLoaders
batch_size = 64

train_dataset = TimeSeriesDataset(X1_train_tensor, X2_train_tensor, y_train_tensor)
test_dataset = TimeSeriesDataset(X1_test_tensor, X2_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, model1, model2):
        super(CombinedModel, self).__init__()
        self.model1 = model1  # HMM for data1
        self.model2 = model2  # LSTM for data2
        self.num_securities = model1.num_securities
        self.num_classes = model1.num_classes
        # Final linear layer to combine the outputs
        self.fc = nn.Linear(self.num_classes * 2, self.num_classes)

    def forward(self, x1, x2):
        # x1: Input for data1 (HMM)
        # x2: Input for data2 (LSTM)
        outputs1 = self.model1(x1)  # (batch_size, num_securities, num_classes)
        outputs2 = self.model2(x2)  # (batch_size, num_securities, num_classes)
        # Concatenate the outputs over the class dimension
        combined_outputs = torch.cat((outputs1, outputs2), dim=2)  # (batch_size, num_securities, num_classes * 2)
        # Pass through the final linear layer
        final_outputs = self.fc(combined_outputs)  # (batch_size, num_securities, num_classes)
        return final_outputs

In [ ]:
def train_combined_model(model, train_loader, criterion, optimizer, num_epochs, device):
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X1_batch, X2_batch, y_batch in train_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X1_batch, X2_batch)
            loss = 0
            for i in range(model.num_securities):
                loss += criterion(outputs[:, i, :], y_batch[:, i])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
def evaluate_combined_model(model, test_loader, device):
    model = model.to(device)
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for X1_batch, X2_batch, y_batch in test_loader:
            X1_batch = X1_batch.to(device)
            X2_batch = X2_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X1_batch, X2_batch)
            _, predicted = torch.max(outputs.data, 2)
            total += y_batch.numel()
            correct += (predicted == y_batch).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

In [ ]:
num_states = 5  # Number of hidden states
hmm_model = HiddenMarkovModel(
    num_states=num_states,
    num_securities=num_securities,
    num_classes=num_classes,
    num_features_per_security=num_features_per_security,
)

# LSTM for data2
input_size_data2 = X2_train_tensor.size(2)  # Number of features for data2
hidden_size = 128
num_layers = 2
lstm_model_data2 = LSTMModelData2(
    input_size=input_size_data2,
    hidden_size=hidden_size,
    num_layers=num_layers,
    num_securities=num_securities,
    num_classes=num_classes,
)

# Combined model
combined_model = CombinedModel(hmm_model, lstm_model_data2)

# Combine parameters for optimization
params = list(combined_model.parameters())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Train the combined model
train_combined_model(combined_model, train_loader, criterion, optimizer, num_epochs, device)
evaluate_combined_model(combined_model, test_loader, device)

Accuracy on test set: 33.37%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Time2Vec implementation
class Time2Vec(nn.Module):
    def __init__(self, seq_len, d_model):
        super(Time2Vec, self).__init__()
        self.seq_len = seq_len
        self.d_model = d_model
        self.w0 = nn.Parameter(torch.randn(1))
        self.b0 = nn.Parameter(torch.randn(1))
        self.w = nn.Parameter(torch.randn(d_model - 1))
        self.b = nn.Parameter(torch.randn(d_model - 1))

    def forward(self, batch_size):
        # Create time indices
        t = torch.arange(self.seq_len).unsqueeze(-1).float().to(self.w0.device)  # Shape: (seq_len, 1)
        # Linear component
        v = self.w0 * t + self.b0  # Shape: (seq_len, 1)
        # Periodic component
        vp = torch.sin(self.w * t + self.b)  # Broadcasting over (seq_len, d_model - 1)
        # Concatenate components
        time_emb = torch.cat([v, vp], dim=-1)  # Shape: (seq_len, d_model)
        # Expand to match batch size
        time_emb = time_emb.unsqueeze(0).expand(batch_size, -1, -1)  # Shape: (batch_size, seq_len, d_model)
        return time_emb

# Corrected Multi-Head Attention Mechanism
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)

        # Perform linear projections and split into heads
        q = self.q_linear(q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1,2)
        k = self.k_linear(k).view(batch_size, -1, self.num_heads, self.d_k).transpose(1,2)
        v = self.v_linear(v).view(batch_size, -1, self.num_heads, self.d_k).transpose(1,2)

        # Calculate attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / np.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn_weights = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn_weights, v)

        # Concatenate heads
        context = context.transpose(1,2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
        output = self.out_linear(context)

        return output

# Corrected Gated Residual Network
class GatedResidualNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GatedResidualNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.elu = nn.ELU()
        self.fc2 = nn.Linear(hidden_size, 2 * input_size)  # Output size doubled for GLU
        self.glu = nn.GLU(dim=-1)  # Splits last dimension into two halves

    def forward(self, x):
        residual = x
        x = self.fc1(x)
        x = self.elu(x)
        x = self.fc2(x)
        x = self.glu(x)
        return residual + x

# Corrected Transformer Encoder Block
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads, hidden_size):
        super(TransformerEncoder, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.grn = GatedResidualNetwork(d_model, hidden_size)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        attn_output = self.attention(x, x, x, mask)
        x = self.norm1(x + attn_output)
        grn_output = self.grn(x)
        x = self.norm2(x + grn_output)
        return x

# Corrected Transformer Decoder Block
class TransformerDecoder(nn.Module):
    def __init__(self, d_model, num_heads, hidden_size):
        super(TransformerDecoder, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.cross_attention = MultiHeadAttention(d_model, num_heads)
        self.grn = GatedResidualNetwork(d_model, hidden_size)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        # Self-attention with masking
        attn_output = self.self_attention(x, x, x, tgt_mask)
        x = self.norm1(x + attn_output)

        # Cross-attention with encoder outputs
        attn_output = self.cross_attention(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + attn_output)

        # Apply GRN and residual connection
        grn_output = self.grn(x)
        x = self.norm3(x + grn_output)
        return x

# Corrected Portfolio Transformer Model
class PortfolioTransformer(nn.Module):
    def __init__(self, num_assets, d_model, num_heads, hidden_size, num_layers, seq_len):
        super(PortfolioTransformer, self).__init__()
        self.num_assets = num_assets
        self.d_model = d_model
        self.seq_len = seq_len
        self.input_proj = nn.Linear(num_assets, d_model)
        self.time2vec = Time2Vec(seq_len, d_model)
        self.encoder_layers = nn.ModuleList(
            [TransformerEncoder(d_model, num_heads, hidden_size) for _ in range(num_layers)]
        )
        self.decoder_layers = nn.ModuleList(
            [TransformerDecoder(d_model, num_heads, hidden_size) for _ in range(num_layers)]
        )
        self.fc_out = nn.Linear(d_model, num_assets)

    def forward(self, x):
        # x shape: (batch_size, seq_len, num_assets)
        batch_size = x.size(0)
        # Project input features to d_model
        x_proj = self.input_proj(x)  # Shape: (batch_size, seq_len, d_model)
        # Get time embeddings
        time_emb = self.time2vec(batch_size)  # Shape: (batch_size, seq_len, d_model)
        # Combine input embeddings and time embeddings
        x = x_proj + time_emb  # Element-wise addition
        # Encoder
        for encoder in self.encoder_layers:
            x = encoder(x)
        # Decoder
        dec_input = x
        for decoder in self.decoder_layers:
            x = decoder(dec_input, x)
        s_i_t = self.fc_out(x)  # Output shape: (batch_size, seq_len, num_assets)
        # Compute weights using the compound function
        weights = torch.sign(s_i_t) * torch.softmax(torch.abs(s_i_t), dim=-1)
        return weights

In [ ]:
import numpy as np
import torch

# Generate random returns data: 100 securities over 1000 time steps
num_securities = 100
num_time_steps = 1000
np.random.seed(42)  # For reproducibility

# Simulate random returns
returns_data = np.random.randn(num_time_steps, num_securities) * 0.01  # Small random returns
prices = 100 + np.cumsum(returns_data, axis=0)  # Simulate price paths

# Convert to PyTorch tensors
prices = torch.tensor(prices, dtype=torch.float32)
returns = torch.tensor(returns_data, dtype=torch.float32)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Prepare the dataset
sequence_length = 20  # Use last 20 days for prediction
X = []
y = []

for i in range(len(prices) - sequence_length):
    X.append(returns[i:i+sequence_length])
    y.append(returns[i+sequence_length])

X = torch.stack(X)  # Shape: (num_samples, seq_len, num_assets)
y = torch.stack(y)  # Shape: (num_samples, num_assets)

dataset = TensorDataset(X, y)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model
num_assets = num_securities
model = PortfolioTransformer(
    num_assets=num_assets, d_model=64, num_heads=8, hidden_size=128, num_layers=4, seq_len=sequence_length
)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, optimizer, data_loader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_X, batch_y in data_loader:
            optimizer.zero_grad()
            portfolio_weights = model(batch_X)  # Output shape: (batch_size, seq_len, num_assets)
            # Use the last time step's weights
            portfolio_weights = portfolio_weights[:, -1, :]  # Shape: (batch_size, num_assets)
            # Compute portfolio returns
            portfolio_returns = torch.sum(portfolio_weights * batch_y, dim=1)
            loss = sharpe_loss(portfolio_returns)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader):.4f}")

# Loss function (Negative Sharpe Ratio)
def sharpe_loss(portfolio_returns, risk_free_rate=0.0):
    # Assuming daily returns, annualize by multiplying by sqrt(252)
    mean_return = torch.mean(portfolio_returns)
    std_return = torch.std(portfolio_returns)
    sharpe_ratio = (mean_return - risk_free_rate) / (std_return + 1e-6)
    return -sharpe_ratio  # Negative for minimization

# Train the model
train(model, optimizer, data_loader, num_epochs=10)

Epoch 1/10, Loss: 0.0155
Epoch 2/10, Loss: -0.1249
Epoch 3/10, Loss: -0.1783
Epoch 4/10, Loss: -0.2097
Epoch 5/10, Loss: -0.2230
Epoch 6/10, Loss: -0.2474
Epoch 7/10, Loss: -0.2255
Epoch 8/10, Loss: -0.2514
Epoch 9/10, Loss: -0.2554
Epoch 10/10, Loss: -0.2540


In [ ]:
## version 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Advanced Time Encoding: Learnable Positional Encoding
class LearnablePositionalEncoding(nn.Module):
    """
    Implements learnable positional encoding for sequences.

    Args:
        seq_len (int): The maximum length of the input sequences.
        d_model (int): The dimension of the model.
    """
    def __init__(self, seq_len, d_model):
        super(LearnablePositionalEncoding, self).__init__()
        self.position_embeddings = nn.Embedding(seq_len, d_model)
        self.seq_len = seq_len

    def forward(self, x):
        """
        Forward pass for positional encoding.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_len, d_model).

        Returns:
            Tensor: Positionally encoded tensor.
        """
        positions = torch.arange(0, self.seq_len, device=x.device).unsqueeze(0)
        pos_embed = self.position_embeddings(positions)
        x = x + pos_embed
        return x

# PortfolioTransformer using PyTorch's nn.Transformer
class PortfolioTransformer(nn.Module):
    """
    Portfolio Transformer model for asset allocation.

    Args:
        num_assets (int): Number of assets in the portfolio.
        d_model (int): Dimension of the model.
        nhead (int): Number of attention heads.
        num_layers (int): Number of transformer encoder layers.
        seq_len (int): Length of the input sequences.
        dropout (float): Dropout probability.
    """
    def __init__(
        self, num_assets, d_model, nhead, num_layers, seq_len, dropout=0.1
    ):
        super(PortfolioTransformer, self).__init__()
        self.num_assets = num_assets
        self.d_model = d_model

        # Input projection layer to map asset returns to model dimension
        self.input_proj = nn.Linear(num_assets, d_model)

        # Learnable positional encoding
        self.pos_encoder = LearnablePositionalEncoding(seq_len, d_model)

        # Transformer encoder layers
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layers, num_layers=num_layers
        )

        # Output layer to map back to asset space
        self.fc_out = nn.Linear(d_model, num_assets)

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        """
        Initializes weights for better convergence.
        """
        initrange = 0.1
        self.input_proj.weight.data.uniform_(-initrange, initrange)
        self.input_proj.bias.data.zero_()
        self.fc_out.bias.data.zero_()
        self.fc_out.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_len, num_assets).

        Returns:
            Tensor: Portfolio weights of shape (batch_size, seq_len, num_assets).
        """
        # Project input to model dimension and scale
        x = self.input_proj(x) * np.sqrt(self.d_model)
        # Add positional encoding
        x = self.pos_encoder(x)
        # Pass through transformer encoder
        x = self.transformer_encoder(x)
        # Map back to asset space
        s_i_t = self.fc_out(x)  # Shape: (batch_size, seq_len, num_assets)
        # Apply activation and normalize weights
        weights = torch.tanh(s_i_t)
        weights = weights / torch.sum(torch.abs(weights), dim=-1, keepdim=True)
        return weights

def sharpe_loss(
    portfolio_returns, portfolio_weights, prev_weights, transaction_cost=0.0002
):
    """
    Computes the negative Sharpe ratio as the loss function, including transaction costs.

    Args:
        portfolio_returns (Tensor): Portfolio returns of shape (batch_size,).
        portfolio_weights (Tensor): Current portfolio weights of shape (batch_size, num_assets).
        prev_weights (Tensor): Previous portfolio weights of shape (batch_size, num_assets).
        transaction_cost (float): Transaction cost rate per unit weight change.

    Returns:
        Tensor: Loss value (negative Sharpe ratio).
    """
    # Compute transaction costs based on weight changes
    tc = transaction_cost * torch.sum(torch.abs(portfolio_weights - prev_weights), dim=1)
    # Net returns after subtracting transaction costs
    net_returns = portfolio_returns - tc
    # Compute mean and standard deviation of returns
    mean_return = torch.mean(net_returns)
    std_return = torch.std(net_returns)
    # Compute Sharpe ratio
    sharpe_ratio = mean_return / (std_return + 1e-6)
    # Return negative Sharpe ratio for minimization
    return -sharpe_ratio

def compute_expected_time_to_target(portfolio_returns, target_sharpe, window=20):
    """
    Estimates the expected time to reach the target Sharpe ratio.

    Args:
        portfolio_returns (list): List of past portfolio returns.
        target_sharpe (float): Target Sharpe ratio.
        window (int): Window size for rolling calculation.

    Returns:
        Tensor: Estimated time to reach the target Sharpe ratio.
    """
    # If insufficient data, return zero
    if len(portfolio_returns) < window:
        return torch.tensor(0.0)
    # Compute rolling mean and std
    recent_returns = torch.tensor(portfolio_returns[-window:])
    rolling_returns = torch.mean(recent_returns)
    rolling_std = torch.std(recent_returns)
    rolling_sharpe = rolling_returns / (rolling_std + 1e-6)
    # Estimate time to target
    if rolling_sharpe >= target_sharpe:
        time_to_target = torch.tensor(0.0)
    else:
        # Simplified estimation
        time_to_target = (target_sharpe - rolling_sharpe) * window
    return time_to_target

def adjust_position_scaling(portfolio_weights, time_to_target, scaling_factor=0.1):
    """
    Adjusts position scaling based on the estimated time to target Sharpe ratio.

    Args:
        portfolio_weights (Tensor): Current portfolio weights.
        time_to_target (Tensor): Estimated time to reach target Sharpe ratio.
        scaling_factor (float): Factor to control scaling sensitivity.

    Returns:
        Tensor: Adjusted portfolio weights.
    """
    # Compute scaling coefficient
    scaling = 1.0 - scaling_factor * torch.sigmoid(time_to_target)
    # Adjust portfolio weights
    adjusted_weights = portfolio_weights * scaling
    return adjusted_weights

def train(model, optimizer, data_loader, num_epochs, target_sharpe=1.0):
    """
    Trains the PortfolioTransformer model.

    Args:
        model (nn.Module): The PortfolioTransformer model.
        optimizer (torch.optim.Optimizer): The optimizer.
        data_loader (DataLoader): DataLoader for training data.
        num_epochs (int): Number of training epochs.
        target_sharpe (float): Target Sharpe ratio for scaling positions.
    """
    model.train()
    portfolio_returns_history = []
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_X, batch_y in data_loader:
            optimizer.zero_grad()
            # Forward pass to get portfolio weights
            portfolio_weights = model(batch_X)
            # Use the last time step's weights
            portfolio_weights = portfolio_weights[:, -1, :]
            # Initialize prev_weights with zeros matching portfolio_weights
            prev_weights = torch.zeros_like(portfolio_weights)
            # Compute portfolio returns
            portfolio_returns = torch.sum(portfolio_weights * batch_y, dim=1)
            # Adjust positions based on expected time to target
            time_to_target = compute_expected_time_to_target(
                portfolio_returns_history, target_sharpe
            )
            portfolio_weights = adjust_position_scaling(
                portfolio_weights, time_to_target
            )
            # Update returns history
            portfolio_returns_history.extend(portfolio_returns.tolist())
            # Compute loss with transaction costs
            loss = sharpe_loss(portfolio_returns, portfolio_weights, prev_weights)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            # Update prev_weights for the next batch
            prev_weights = portfolio_weights.detach()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader):.4f}")

def backtest(model, data, train_size, test_size, sequence_length, target_sharpe=1.0):
    """
    Performs backtesting by training on a subset of data and testing on future data.

    Args:
        model (nn.Module): The PortfolioTransformer model.
        data (Tensor): Asset returns data.
        train_size (int): Number of samples for training.
        test_size (int): Number of samples for testing.
        sequence_length (int): Length of input sequences.
        target_sharpe (float): Target Sharpe ratio for scaling positions.
    """
    num_samples = len(data) - sequence_length
    train_indices = range(train_size)
    test_indices = range(train_size, train_size + test_size)

    # Prepare training data
    X_train = []
    y_train = []
    for i in train_indices:
        X_train.append(data[i:i+sequence_length])
        y_train.append(data[i+sequence_length])
    X_train = torch.stack(X_train)
    y_train = torch.stack(y_train)
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=False)

    # Prepare test data
    X_test = []
    y_test = []
    for i in test_indices:
        X_test.append(data[i:i+sequence_length])
        y_test.append(data[i+sequence_length])
    X_test = torch.stack(X_test)
    y_test = torch.stack(y_test)

    # Train the model
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    train(model, optimizer, train_loader, num_epochs=10, target_sharpe=target_sharpe)

    # Backtest on test data
    model.eval()
    prev_weights = torch.zeros(y_test.size(1))
    portfolio_returns_history = []
    with torch.no_grad():
        for i in range(len(X_test)):
            batch_X = X_test[i:i+1]  # Shape: (1, seq_len, num_assets)
            batch_y = y_test[i:i+1]  # Shape: (1, num_assets)
            # Get portfolio weights
            portfolio_weights = model(batch_X)
            portfolio_weights = portfolio_weights[:, -1, :]
            # Compute portfolio returns
            portfolio_returns = torch.sum(portfolio_weights * batch_y, dim=1)
            # Adjust positions based on expected time to target
            time_to_target = compute_expected_time_to_target(
                portfolio_returns_history, target_sharpe=target_sharpe
            )
            portfolio_weights = adjust_position_scaling(
                portfolio_weights, time_to_target
            )
            # Update returns history
            portfolio_returns_history.extend(portfolio_returns.tolist())
            # Update prev_weights
            prev_weights = portfolio_weights.detach()
        # Compute cumulative returns and Sharpe ratio
        portfolio_returns = np.array(portfolio_returns_history)
        cumulative_returns = np.cumprod(1 + portfolio_returns) - 1
        mean_return = np.mean(portfolio_returns)
        std_return = np.std(portfolio_returns)
        sharpe_ratio = mean_return / (std_return + 1e-6) * np.sqrt(252)
        print(f"Cumulative Return: {cumulative_returns[-1]:.4f}")
        print(f"Annualized Sharpe Ratio: {sharpe_ratio:.4f}")

In [ ]:
import numpy as np
import torch

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Generate random returns data: 100 securities over 1000 time steps
num_securities = 100
num_time_steps = 1000

# Simulate random returns (e.g., normally distributed with small mean and std)
returns_data = np.random.normal(loc=0.001, scale=0.01, size=(num_time_steps, num_securities))
returns = torch.tensor(returns_data, dtype=torch.float32)

In [ ]:
# Define parameters
sequence_length = 20
train_size = 700
test_size = 200
num_assets = num_securities

# Initialize the model
model = PortfolioTransformer(
    num_assets=num_assets,
    d_model=64,
    nhead=8,
    num_layers=4,
    seq_len=sequence_length,
    dropout=0.1
)

# Run backtest
backtest(model, returns, train_size, test_size, sequence_length, target_sharpe=1.0)

Epoch 1/10, Loss: -0.4861
Epoch 2/10, Loss: -0.7983
Epoch 3/10, Loss: -0.8931
Epoch 4/10, Loss: -0.9067
Epoch 5/10, Loss: -0.9006
Epoch 6/10, Loss: -0.9224
Epoch 7/10, Loss: -0.9115
Epoch 8/10, Loss: -0.9196
Epoch 9/10, Loss: -0.9191
Epoch 10/10, Loss: -0.9201
Cumulative Return: 0.2299
Annualized Sharpe Ratio: 12.6064


In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from torch.utils.tensorboard import SummaryWriter
import time

# Parameters
num_timesteps = 5000  # Total number of time steps
num_assets = 10       # Number of assets in the portfolio
seq_len = 20          # Length of input sequences
d_model = 64          # Model dimension for Transformer
batch_size = 64       # Batch size for training
num_epochs = 5        # Number of training epochs

# Generate synthetic returns data
np.random.seed(42)
returns_data = np.random.normal(0, 0.01, size=(num_timesteps, num_assets)).astype(np.float32)
returns_data = torch.tensor(returns_data)

# Dataset and DataLoader for training
class ReturnsDataset(Dataset):
    """
    Dataset for asset returns.

    Args:
        data (Tensor): Asset returns data of shape (num_timesteps, num_assets).
        seq_len (int): Length of input sequences.
    """
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        X = self.data[idx:idx + self.seq_len]
        y = self.data[idx + self.seq_len]
        return X, y

# Split data into training and testing sets
train_ratio = 0.8
train_size = int(len(returns_data) * train_ratio)
train_data = returns_data[:train_size]
test_data = returns_data[train_size:]

train_dataset = ReturnsDataset(train_data, seq_len)
test_dataset = ReturnsDataset(test_data, seq_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Loss function: Negative Sharpe Ratio
def sharpe_loss(portfolio_returns, portfolio_weights, prev_weights, transaction_cost=0.0002):
    """
    Computes the negative Sharpe ratio as the loss function, including transaction costs.

    Args:
        portfolio_returns (Tensor): Portfolio returns of shape (batch_size,).
        portfolio_weights (Tensor): Current portfolio weights of shape (batch_size, num_assets).
        prev_weights (Tensor): Previous portfolio weights of shape (batch_size, num_assets).
        transaction_cost (float): Transaction cost rate per unit weight change.

    Returns:
        Tensor: Loss value (negative Sharpe ratio).
    """
    # Compute transaction costs based on weight changes
    tc = transaction_cost * torch.sum(torch.abs(portfolio_weights - prev_weights), dim=1)
    # Net returns after subtracting transaction costs
    net_returns = portfolio_returns - tc
    # Compute mean and standard deviation of returns
    mean_return = torch.mean(net_returns)
    std_return = torch.std(net_returns)
    # Compute Sharpe ratio
    sharpe_ratio = mean_return / (std_return + 1e-6)
    # Return negative Sharpe ratio for minimization
    return -sharpe_ratio

# Base class for models
class BaseModel(nn.Module):
    """
    Base class for asset allocation models.

    Args:
        num_assets (int): Number of assets in the portfolio.
    """
    def __init__(self, num_assets):
        super(BaseModel, self).__init__()
        self.num_assets = num_assets

    def forward(self, x):
        """
        Forward pass.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_len, num_assets).

        Returns:
            Tensor: Portfolio weights of shape (batch_size, num_assets).
        """
        raise NotImplementedError

# Logistic Regression Model
class LogisticRegressionModel(BaseModel):
    """
    Logistic Regression Model for asset allocation.

    Args:
        input_size (int): Size of input features (seq_len * num_assets).
        num_assets (int): Number of assets.
    """
    def __init__(self, input_size, num_assets):
        super(LogisticRegressionModel, self).__init__(num_assets)
        self.linear = nn.Linear(input_size, num_assets)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        out = self.linear(x)
        weights = torch.tanh(out)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# LSTM Model
class LSTMModel(BaseModel):
    """
    LSTM Model for asset allocation.

    Args:
        num_assets (int): Number of assets.
        hidden_size (int): Size of hidden state.
        num_layers (int): Number of LSTM layers.
    """
    def __init__(self, num_assets, hidden_size=64, num_layers=2):
        super(LSTMModel, self).__init__(num_assets)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=num_assets, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_assets)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        weights = torch.tanh(out)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# CNN Model
class CNNModel(BaseModel):
    """
    CNN Model for asset allocation.

    Args:
        num_assets (int): Number of assets.
    """
    def __init__(self, num_assets):
        super(CNNModel, self).__init__(num_assets)
        self.conv1 = nn.Conv1d(in_channels=num_assets, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.fc = nn.Linear(64, num_assets)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch_size, num_assets, seq_len)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        weights = torch.tanh(x)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# Transformer Model
class TransformerModel(BaseModel):
    """
    Transformer Model for asset allocation.

    Args:
        num_assets (int): Number of assets.
        d_model (int): Dimension of the model.
        nhead (int): Number of attention heads.
        num_layers (int): Number of transformer encoder layers.
        seq_len (int): Length of the input sequences.
        dropout (float): Dropout probability.
    """
    def __init__(self, num_assets, d_model=64, nhead=4, num_layers=2, seq_len=20, dropout=0.1):
        super(TransformerModel, self).__init__(num_assets)
        self.d_model = d_model
        self.input_proj = nn.Linear(num_assets, d_model)
        self.pos_encoder = LearnablePositionalEncoding(seq_len, d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, num_assets)
        self._init_weights()

    def _init_weights(self):
        initrange = 0.1
        self.input_proj.weight.data.uniform_(-initrange, initrange)
        self.input_proj.bias.data.zero_()
        self.fc_out.bias.data.zero_()
        self.fc_out.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        x = self.input_proj(x) * np.sqrt(self.d_model)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]  # Last time step
        x = self.fc_out(x)
        weights = torch.tanh(x)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# Learnable Positional Encoding
class LearnablePositionalEncoding(nn.Module):
    """
    Implements learnable positional encoding for sequences.

    Args:
        seq_len (int): The maximum length of the input sequences.
        d_model (int): The dimension of the model.
    """
    def __init__(self, seq_len, d_model):
        super(LearnablePositionalEncoding, self).__init__()
        self.position_embeddings = nn.Embedding(seq_len, d_model)
        self.seq_len = seq_len

    def forward(self, x):
        positions = torch.arange(0, self.seq_len, device=x.device).unsqueeze(0)
        pos_embed = self.position_embeddings(positions)
        x = x + pos_embed
        return x

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs, device):
    """
    Training loop for the model.

    Args:
        model (nn.Module): The model to train.
        train_loader (DataLoader): DataLoader for training data.
        criterion (function): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer.
        num_epochs (int): Number of epochs.
        device (torch.device): Computation device.
    """
    model = model.to(device)
    prev_weights = torch.zeros((batch_size, model.num_assets)).to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            # Forward pass
            portfolio_weights = model(X_batch)
            # Compute portfolio returns
            portfolio_returns = torch.sum(portfolio_weights * y_batch, dim=1)
            # Compute loss
            loss = criterion(portfolio_returns, portfolio_weights, prev_weights[:portfolio_weights.size(0)])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            # Update prev_weights
            prev_weights[:portfolio_weights.size(0)] = portfolio_weights.detach()
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}')

# Backtesting function
def backtest_model(model, test_loader, device):
    """
    Backtesting the model on test data.

    Args:
        model (nn.Module): The trained model.
        test_loader (DataLoader): DataLoader for test data.
        device (torch.device): Computation device.
    """
    model = model.to(device)
    model.eval()
    portfolio_returns_history = []
    prev_weights = torch.zeros((1, model.num_assets)).to(device)
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            portfolio_weights = model(X_batch)
            portfolio_returns = torch.sum(portfolio_weights * y_batch, dim=1)
            portfolio_returns_history.extend(portfolio_returns.cpu().numpy())
            # Update prev_weights
            prev_weights = portfolio_weights.detach()
    portfolio_returns = np.array(portfolio_returns_history)
    cumulative_returns = np.cumprod(1 + portfolio_returns) - 1
    mean_return = np.mean(portfolio_returns)
    std_return = np.std(portfolio_returns)
    sharpe_ratio = mean_return / (std_return + 1e-6) * np.sqrt(252)
    print(f"Cumulative Return: {cumulative_returns[-1]:.6f}")
    print(f"Annualized Sharpe Ratio: {sharpe_ratio:.6f}")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Example usage with Logistic Regression Model
print("\nTraining Logistic Regression Model")
model = LogisticRegressionModel(input_size=seq_len * num_assets, num_assets=num_assets)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)

# Example usage with LSTM Model
print("\nTraining LSTM Model")
model = LSTMModel(num_assets=num_assets)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)

# Example usage with CNN Model
print("\nTraining CNN Model")
model = CNNModel(num_assets=num_assets)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)

# Example usage with Transformer Model
print("\nTraining Transformer Model")
model = TransformerModel(num_assets=num_assets, d_model=d_model, seq_len=seq_len)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)

# New Model provided by the user
print("\nTraining Portfolio Transformer Model")
model = PortfolioTransformer(num_assets=num_assets, d_model=d_model, nhead=4, num_layers=2, seq_len=seq_len)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)


Training Logistic Regression Model
Epoch [1/5], Loss: 0.009969
Epoch [2/5], Loss: -0.013706
Epoch [3/5], Loss: -0.030145
Epoch [4/5], Loss: -0.055203
Epoch [5/5], Loss: -0.082532
Cumulative Return: -0.071690
Annualized Sharpe Ratio: -0.277367

Training LSTM Model
Epoch [1/5], Loss: 0.015314
Epoch [2/5], Loss: -0.019160
Epoch [3/5], Loss: -0.023031
Epoch [4/5], Loss: -0.035522
Epoch [5/5], Loss: -0.023104
Cumulative Return: -0.001735
Annualized Sharpe Ratio: 0.024882

Training CNN Model
Epoch [1/5], Loss: 0.008797
Epoch [2/5], Loss: -0.006740
Epoch [3/5], Loss: -0.012274
Epoch [4/5], Loss: -0.018750
Epoch [5/5], Loss: -0.021238
Cumulative Return: 0.168103
Annualized Sharpe Ratio: 0.680245

Training Transformer Model
Epoch [1/5], Loss: 0.055456
Epoch [2/5], Loss: 0.000523
Epoch [3/5], Loss: -0.012663
Epoch [4/5], Loss: -0.023768
Epoch [5/5], Loss: -0.035341
Cumulative Return: -0.019282
Annualized Sharpe Ratio: -0.069464

Training Portfolio Transformer Model


RuntimeError: The size of tensor a (20) must match the size of tensor b (64) at non-singleton dimension 1

In [25]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from torch.utils.tensorboard import SummaryWriter
import time

# Parameters
num_timesteps = 5000  # Total number of time steps
num_assets = 10       # Number of assets in the portfolio
seq_len = 20          # Length of input sequences
d_model = 64          # Model dimension for Transformer
batch_size = 64       # Batch size for training
num_epochs = 5        # Number of training epochs

# Generate synthetic returns data
np.random.seed(42)
returns_data = np.random.normal(0, 0.01, size=(num_timesteps, num_assets)).astype(np.float32)
returns_data = torch.tensor(returns_data)

# Dataset and DataLoader for training
class ReturnsDataset(Dataset):
    """
    Dataset for asset returns.

    Args:
        data (Tensor): Asset returns data of shape (num_timesteps, num_assets).
        seq_len (int): Length of input sequences.
    """
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        X = self.data[idx:idx + self.seq_len]
        y = self.data[idx + self.seq_len]
        return X, y

# Split data into training and testing sets
train_ratio = 0.8
train_size = int(len(returns_data) * train_ratio)
train_data = returns_data[:train_size]
test_data = returns_data[train_size:]

train_dataset = ReturnsDataset(train_data, seq_len)
test_dataset = ReturnsDataset(test_data, seq_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # Use batch_size=1 for backtesting

# Loss function: Negative Sharpe Ratio
def sharpe_loss(portfolio_returns, portfolio_weights, prev_weights, transaction_cost=0.0002):
    """
    Computes the negative Sharpe ratio as the loss function, including transaction costs.

    Args:
        portfolio_returns (Tensor): Portfolio returns of shape (batch_size,).
        portfolio_weights (Tensor): Current portfolio weights of shape (batch_size, num_assets).
        prev_weights (Tensor): Previous portfolio weights of shape (batch_size, num_assets).
        transaction_cost (float): Transaction cost rate per unit weight change.

    Returns:
        Tensor: Loss value (negative Sharpe ratio).
    """
    # Compute transaction costs based on weight changes
    tc = transaction_cost * torch.sum(torch.abs(portfolio_weights - prev_weights), dim=1)
    # Net returns after subtracting transaction costs
    net_returns = portfolio_returns - tc
    # Compute mean and standard deviation of returns
    mean_return = torch.mean(net_returns)
    std_return = torch.std(net_returns)
    # Compute Sharpe ratio
    sharpe_ratio = mean_return / (std_return + 1e-6)
    # Return negative Sharpe ratio for minimization
    return -sharpe_ratio

# Base class for models
class BaseModel(nn.Module):
    """
    Base class for asset allocation models.

    Args:
        num_assets (int): Number of assets in the portfolio.
    """
    def __init__(self, num_assets):
        super(BaseModel, self).__init__()
        self.num_assets = num_assets

    def forward(self, x):
        """
        Forward pass.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_len, num_assets).

        Returns:
            Tensor: Portfolio weights of shape (batch_size, num_assets).
        """
        raise NotImplementedError

# Logistic Regression Model
class LogisticRegressionModel(BaseModel):
    """
    Logistic Regression Model for asset allocation.

    Args:
        input_size (int): Size of input features (seq_len * num_assets).
        num_assets (int): Number of assets.
    """
    def __init__(self, input_size, num_assets):
        super(LogisticRegressionModel, self).__init__(num_assets)
        self.linear = nn.Linear(input_size, num_assets)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten
        out = self.linear(x)
        weights = torch.tanh(out)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# LSTM Model
class LSTMModel(BaseModel):
    """
    LSTM Model for asset allocation.

    Args:
        num_assets (int): Number of assets.
        hidden_size (int): Size of hidden state.
        num_layers (int): Number of LSTM layers.
    """
    def __init__(self, num_assets, hidden_size=64, num_layers=2):
        super(LSTMModel, self).__init__(num_assets)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=num_assets, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_assets)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        weights = torch.tanh(out)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# CNN Model
class CNNModel(BaseModel):
    """
    CNN Model for asset allocation.

    Args:
        num_assets (int): Number of assets.
    """
    def __init__(self, num_assets):
        super(CNNModel, self).__init__(num_assets)
        self.conv1 = nn.Conv1d(in_channels=num_assets, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.fc = nn.Linear(64, num_assets)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch_size, num_assets, seq_len)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        weights = torch.tanh(x)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# Transformer Model
class TransformerModel(BaseModel):
    """
    Transformer Model for asset allocation.

    Args:
        num_assets (int): Number of assets.
        d_model (int): Dimension of the model.
        nhead (int): Number of attention heads.
        num_layers (int): Number of transformer encoder layers.
        seq_len (int): Length of the input sequences.
        dropout (float): Dropout probability.
    """
    def __init__(self, num_assets, d_model=64, nhead=4, num_layers=2, seq_len=20, dropout=0.1):
        super(TransformerModel, self).__init__(num_assets)
        self.d_model = d_model
        self.input_proj = nn.Linear(num_assets, d_model)
        self.pos_encoder = LearnablePositionalEncoding(seq_len, d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, num_assets)
        self._init_weights()

    def _init_weights(self):
        initrange = 0.1
        self.input_proj.weight.data.uniform_(-initrange, initrange)
        self.input_proj.bias.data.zero_()
        self.fc_out.bias.data.zero_()
        self.fc_out.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        x = self.input_proj(x) * np.sqrt(self.d_model)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        x = self.transformer_encoder(x)
        x = x.permute(1, 0, 2)  # (batch_size, seq_len, d_model)
        x = x[:, -1, :]  # Last time step
        x = self.fc_out(x)
        weights = torch.tanh(x)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# Learnable Positional Encoding
class LearnablePositionalEncoding(nn.Module):
    """
    Implements learnable positional encoding for sequences.

    Args:
        seq_len (int): The maximum length of the input sequences.
        d_model (int): The dimension of the model.
    """
    def __init__(self, seq_len, d_model):
        super(LearnablePositionalEncoding, self).__init__()
        self.position_embeddings = nn.Embedding(seq_len, d_model)
        self.seq_len = seq_len

    def forward(self, x):
        positions = torch.arange(0, x.size(1), device=x.device).unsqueeze(0).long()
        pos_embed = self.position_embeddings(positions)
        x = x + pos_embed
        return x

# Portfolio Transformer Model
class PortfolioTransformer(BaseModel):
    """
    Portfolio Transformer model for asset allocation.

    Args:
        num_assets (int): Number of assets in the portfolio.
        d_model (int): Dimension of the model.
        nhead (int): Number of attention heads.
        num_layers (int): Number of transformer encoder layers.
        seq_len (int): Length of the input sequences.
        dropout (float): Dropout probability.
    """
    def __init__(
        self, num_assets, d_model, nhead, num_layers, seq_len, dropout=0.1
    ):
        super(PortfolioTransformer, self).__init__(num_assets)
        self.d_model = d_model

        # Input projection layer to map asset returns to model dimension
        self.input_proj = nn.Linear(num_assets, d_model)

        # Learnable positional encoding
        self.pos_encoder = LearnablePositionalEncoding(seq_len, d_model)

        # Transformer encoder layers
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layers, num_layers=num_layers
        )

        # Output layer to map back to asset space
        self.fc_out = nn.Linear(d_model, num_assets)

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        """
        Initializes weights for better convergence.
        """
        initrange = 0.1
        self.input_proj.weight.data.uniform_(-initrange, initrange)
        self.input_proj.bias.data.zero_()
        self.fc_out.bias.data.zero_()
        self.fc_out.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_len, num_assets).

        Returns:
            Tensor: Portfolio weights of shape (batch_size, num_assets).
        """
        # Project input to model dimension and scale
        x = self.input_proj(x) * np.sqrt(self.d_model)
        # Add positional encoding
        x = self.pos_encoder(x)
        # Permute dimensions to match expected input of TransformerEncoder
        x = x.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        # Pass through transformer encoder
        x = self.transformer_encoder(x)
        # Permute back to (batch_size, seq_len, d_model)
        x = x.permute(1, 0, 2)
        # Get the last time step
        x = x[:, -1, :]  # (batch_size, d_model)
        # Map back to asset space
        s_i_t = self.fc_out(x)  # Shape: (batch_size, num_assets)
        # Apply activation and normalize weights
        weights = torch.tanh(s_i_t)
        weights = weights / torch.sum(torch.abs(weights), dim=1, keepdim=True)
        return weights

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs, device):
    """
    Training loop for the model.

    Args:
        model (nn.Module): The model to train.
        train_loader (DataLoader): DataLoader for training data.
        criterion (function): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer.
        num_epochs (int): Number of epochs.
        device (torch.device): Computation device.
    """
    model = model.to(device)
    prev_weights = torch.zeros((batch_size, model.num_assets)).to(device)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            # Forward pass
            portfolio_weights = model(X_batch)
            # Compute portfolio returns
            portfolio_returns = torch.sum(portfolio_weights * y_batch, dim=1)
            # Compute loss
            loss = criterion(portfolio_returns, portfolio_weights, prev_weights[:portfolio_weights.size(0)])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            # Update prev_weights
            prev_weights[:portfolio_weights.size(0)] = portfolio_weights.detach()
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}')

# Backtesting function
def backtest_model(model, test_loader, device):
    """
    Backtesting the model on test data.

    Args:
        model (nn.Module): The trained model.
        test_loader (DataLoader): DataLoader for test data.
        device (torch.device): Computation device.
    """
    model = model.to(device)
    model.eval()
    portfolio_returns_history = []
    prev_weights = torch.zeros((1, model.num_assets)).to(device)
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            portfolio_weights = model(X_batch)
            portfolio_returns = torch.sum(portfolio_weights * y_batch, dim=1)
            portfolio_returns_history.extend(portfolio_returns.cpu().numpy())
            # Update prev_weights
            prev_weights = portfolio_weights.detach()
    portfolio_returns = np.array(portfolio_returns_history)
    cumulative_returns = np.cumprod(1 + portfolio_returns) - 1
    mean_return = np.mean(portfolio_returns)
    std_return = np.std(portfolio_returns)
    sharpe_ratio = mean_return / (std_return + 1e-6) * np.sqrt(252)
    print(f"Cumulative Return: {cumulative_returns[-1]:.6f}")
    print(f"Annualized Sharpe Ratio: {sharpe_ratio:.6f}")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Example usage with Logistic Regression Model
print("\nTraining Logistic Regression Model")
model = LogisticRegressionModel(input_size=seq_len * num_assets, num_assets=num_assets)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)

# Example usage with LSTM Model
print("\nTraining LSTM Model")
model = LSTMModel(num_assets=num_assets)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)

# Example usage with CNN Model
print("\nTraining CNN Model")
model = CNNModel(num_assets=num_assets)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)

# Example usage with Transformer Model
print("\nTraining Transformer Model")
model = TransformerModel(num_assets=num_assets, d_model=d_model, seq_len=seq_len)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)

# New Model provided by the user
print("\nTraining Portfolio Transformer Model")
model = PortfolioTransformer(num_assets=num_assets, d_model=d_model, nhead=4, num_layers=2, seq_len=seq_len)
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, sharpe_loss, optimizer, num_epochs, device)
backtest_model(model, test_loader, device)


Training Logistic Regression Model
Epoch [1/5], Loss: 0.032195
Epoch [2/5], Loss: -0.005823
Epoch [3/5], Loss: -0.027968
Epoch [4/5], Loss: -0.059304
Epoch [5/5], Loss: -0.124792
Cumulative Return: 0.098651
Annualized Sharpe Ratio: 0.424042

Training LSTM Model
Epoch [1/5], Loss: 0.000014
Epoch [2/5], Loss: -0.012548
Epoch [3/5], Loss: -0.020844
Epoch [4/5], Loss: -0.017139
Epoch [5/5], Loss: -0.019319
Cumulative Return: -0.051313
Annualized Sharpe Ratio: -0.166005

Training CNN Model
Epoch [1/5], Loss: 0.009937
Epoch [2/5], Loss: 0.006323
Epoch [3/5], Loss: 0.004441
Epoch [4/5], Loss: -0.014695
Epoch [5/5], Loss: -0.013243
Cumulative Return: 0.122629
Annualized Sharpe Ratio: 0.562786

Training Transformer Model


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch [1/5], Loss: -0.001458
Epoch [2/5], Loss: -0.026020
Epoch [3/5], Loss: -0.030113
Epoch [4/5], Loss: -0.025826
Epoch [5/5], Loss: -0.024897
Cumulative Return: 0.040647
Annualized Sharpe Ratio: 0.229768

Training Portfolio Transformer Model
Epoch [1/5], Loss: 0.006854
Epoch [2/5], Loss: -0.027711
Epoch [3/5], Loss: -0.010189
Epoch [4/5], Loss: -0.016794
Epoch [5/5], Loss: -0.026215
Cumulative Return: -0.077065
Annualized Sharpe Ratio: -0.370159
